# Data:

In [1]:
using Zygote
using JuMP
using Ipopt
using Gurobi
using LinearAlgebra
using MathOptInterface
const MOI = MathOptInterface
using SparseArrays

In [2]:
# %CASE9    Power flow data for 9 bus, 3 generator case.
# %   Please see CASEFORMAT for details on the case file format.
# %
# %   Based on data from Joe H. Chow's book, p. 70.

# %   MATPOWER

# %% MATPOWER Case Format : Version 2
# mpc.version = '2';

# %%-----  Power Flow Data  -----%%
# %% system MVA base
# mpc.baseMVA = 100;

# %% bus data
# %	bus_i	type	Pd	Qd	Gs	Bs	area	Vm	Va	baseKV	zone	Vmax	Vmin
bus = [
	1	3	0	0	0	0	1	1	0	345	1	1.1	0.9;
	2	2	0	0	0	0	1	1	0	345	1	1.1	0.9;
	3	2	0	0	0	0	1	1	0	345	1	1.1	0.9;
	4	1	0	0	0	0	1	1	0	345	1	1.1	0.9;
	5	1	90	30	0	0	1	1	0	345	1	1.1	0.9;
	6	1	0	0	0	0	1	1	0	345	1	1.1	0.9;
	7	1	100	35	0	0	1	1	0	345	1	1.1	0.9;
	8	1	0	0	0	0	1	1	0	345	1	1.1	0.9;
	9	1	125	50	0	0	1	1	0	345	1	1.1	0.9;
];

# %% generator data
# %	bus	Pg	Qg	Qmax	Qmin	Vg	mBase	status	Pmax	Pmin	Pc1	Pc2	Qc1min	Qc1max	Qc2min	Qc2max	ramp_agc	ramp_10	ramp_30	ramp_q	apf
gen = [
	1	0	0	300	-300	1	100	1	250	10	0	0	0	0	0	0	0	0	0	0	0;
	2	163	0	300	-300	1	100	1	300	10	0	0	0	0	0	0	0	0	0	0	0;
	3	85	0	300	-300	1	100	1	270	10	0	0	0	0	0	0	0	0	0	0	0;
];

# %% branch data
# %	fbus	tbus	r	x	b	rateA	rateB	rateC	ratio	angle	status	angmin	angmax
branch = [
	1	4	0	0.0576	0	250	250	250	0	0	1	-360	360;
	4	5	0.017	0.092	0.158	250	250	250	0	0	1	-360	360;
	5	6	0.039	0.17	0.358	150	150	150	0	0	1	-360	360;
	3	6	0	0.0586	0	300	300	300	0	0	1	-360	360;
	6	7	0.0119	0.1008	0.209	150	150	150	0	0	1	-360	360;
	7	8	0.0085	0.072	0.149	250	250	250	0	0	1	-360	360;
	8	2	0	0.0625	0	250	250	250	0	0	1	-360	360;
	8	9	0.032	0.161	0.306	250	250	250	0	0	1	-360	360;
	9	4	0.01	0.085	0.176	250	250	250	0	0	1	-360	360;
];

# %%-----  OPF Data  -----%%
# %% generator cost data
# %	1	startup	shutdown	n	x1	y1	...	xn	yn
# %	2	startup	shutdown	n	c(n-1)	...	c0
gencost = [
	2	1500	0	3	0.11	5	150;
	2	2000	0	3	0.085	1.2	600;
	2	3000	0	3	0.1225	1	335;
];

# Model:

In [3]:
## extract data from file

# user defined sets:
E_b_f = Dict(1=>[4], 2=>[], 3=>[6],4=>[5],5=>[6], 6=>[7], 7=>[8], 8=>[2,9], 9=>[4]) # forward branches (from)
E_b_t = Dict(1 =>[], 2=>[8], 3=>[],4=>[1,9],5=>[4],6=>[3,5],7=>[6],8=>[7],9=>[8]) # backward branches (to)
β_g = Dict(1 =>[1], 2=>[2], 3=>[3],4=>[],5=>[],6=>[],7=>[],8=>[],9=>[])  # generator locations (at which buses)

G = [x for x in 1:size(gen,1)] # generators
B = [x for x in 1:size(bus,1)] # buses

@assert sum(length(v) for (k,v) in E_b_f) == sum(length(v) for (k,v) in E_b_t)

ll = sum(length(v) for (k,v) in E_b_f)
lg = length(G)
lb = length(B)

gen_c = Dict()
for g in G
    gen_c[g] = gencost[g,5:7]
end

# calculate sus and con
line_con = Dict()
line_sus = Dict()
line_therm_lim = Dict()
for i in 1:size(branch,1) # iter over rows of branch
    fbus = branch[i,1]; tbus = branch[i,2]
    conduct = branch[i, 3] / ((branch[i, 3])^2 + (branch[i, 4])^2)
    sus = -branch[i, 4] / ((branch[i, 3])^2 + (branch[i, 4])^2)
    
    line_con[(fbus,tbus)] = conduct
    line_sus[(fbus,tbus)] = sus
    
    line_con[(tbus,fbus)] = conduct
    line_sus[(tbus,fbus)] = sus
    
    line_therm_lim[(fbus,tbus)] = branch[i,6]
    line_therm_lim[(tbus,fbus)] = branch[i,6]
end


# extracting data from data file
bus_types = bus[:, 2]
demand_real = 0.01*bus[:,3]
demand_reactive = 0.01*bus[:, 4]
shunt_conductance = bus[:, 5]
shunt_susceptance = bus[:, 6]
pg_minus = 0.01*gen[:,10]
pg_plus = 0.01*gen[:,9]
qg_minus = 0.01*gen[:, 5]
qg_plus = 0.01*gen[:, 4]
vb_minus = bus[:, 13]
vb_plus = bus[:, 12];

# Feasibility restoration phase

In [5]:
function restoration_phase(pk, qk, vk, δk, pbbfk, pbbtk, qbbfk, qbbtk, ρ)
    println("Solving restoration problem...")
    
    is_s_eq_0 = false
    s_not_0 = false
    slack_list = []
    
    nlmod = Model(Ipopt.Optimizer)
    set_string_names_on_creation(nlmod, false)
    set_silent(nlmod)
    
    @variable(nlmod, pg[g in G]) # real power generation
    @variable(nlmod, qg[g in G]) # reactive power generation
    @variable(nlmod, vb[b in B],(start=1.0)) # bus voltage
    @variable(nlmod, δb[b in B],(start=0.0)) # bus voltage phase
    @variable(nlmod, pbbf[b1 in B, b2 in E_b_f[b1]]) # line real power (from)
    @variable(nlmod, pbbt[b1 in B, b2 in E_b_f[b1]]) # line real power (to)
    @variable(nlmod, qbbf[b1 in B, b2 in E_b_f[b1]]) # line reactive power (from)
    @variable(nlmod, qbbt[b1 in B, b2 in E_b_f[b1]]) # line reactive power (to)

    
    @NLobjective(nlmod, Min, sum(100^2*gen_c[g][1] * pg[g]^2 + 100*gen_c[g][2] * pg[g] + gen_c[g][3] for g in G))
    
    # kcl for real power
    for b in B
        @NLconstraint(nlmod, sum(pg[g] for g in β_g[b]) == demand_real[b] +
                                                           sum(pbbf[b,b_] for b_ in E_b_f[b]) +
                                                           sum(pbbt[b_,b] for b_ in E_b_t[b]) +
                                                           shunt_conductance[b] * (vb[b])^2)
    end
    
    # kcl for reactive power
    for b in B
        @NLconstraint(nlmod, sum(qg[g] for g in β_g[b]) == demand_reactive[b] + sum(qbbf[b,b_] for b_ in E_b_f[b]) +
                                sum(qbbt[b_,b] for b_ in E_b_t[b]) - shunt_susceptance[b] * (vb[b])^2)
    end
    

    # kvl real power from
    for b1 in B
        for b2 in E_b_f[b1]
            @NLconstraint(nlmod, line_con[(b1,b2)]* (vb[b1])^2 - vb[b1]*vb[b2]*
                    (line_con[(b1,b2)] * cos(δb[b1] - δb[b2]) + line_sus[(b1,b2)] * sin(δb[b1] - δb[b2])) == pbbf[b1, b2])
        end
    end
    
    # kvl real power to
    for b1 in B
        for b2 in E_b_f[b1]
            @NLconstraint(nlmod, line_con[(b2,b1)]* (vb[b2])^2 - vb[b2]*vb[b1]*
                    (line_con[(b2,b1)] * cos(δb[b2] - δb[b1]) + line_sus[(b2,b1)] * sin(δb[b2] - δb[b1])) == pbbt[b1, b2])
        end
    end
    
    # kvl reactive power from
    for b1 in B
        for b2 in E_b_f[b1]
            @NLconstraint(nlmod, -line_sus[(b1,b2)]* (vb[b1])^2 - vb[b1]*vb[b2]*
                        (line_con[(b1,b2)] * sin(δb[b1] - δb[b2]) - line_sus[(b1,b2)] * cos(δb[b1] - δb[b2])) == qbbf[b1, b2])
        end
    end
        
    # kvl reactive power to
    for b1 in B
        for b2 in E_b_f[b1]
            @NLconstraint(nlmod, -line_sus[(b2,b1)]* (vb[b2])^2 - vb[b2]*vb[b1]*
                        (line_con[(b2,b1)] * sin(δb[b2] - δb[b1]) - line_sus[(b2,b1)] * cos(δb[b2] - δb[b1])) == qbbt[b1, b2])
        end
    end
    
    # thermal limits from
    therm_lim_ctr = 0
    for b1 in B
        for b2 in E_b_f[b1]
            if line_therm_lim[(b1, b2)] != 0.0
                @NLconstraint(nlmod, (pbbf[b1, b2])^2 + (qbbf[b1, b2])^2 <= (0.01*line_therm_lim[(b1, b2)])^2)
                therm_lim_ctr += 1
            end
        end   
    end
    
    # thermal limits to
    for b1 in B
        for b2 in E_b_f[b1]
            if line_therm_lim[(b1, b2)] != 0.0
                @NLconstraint(nlmod, (pbbt[b1, b2])^2 + (qbbt[b1, b2])^2 <= (0.01*line_therm_lim[(b1, b2)])^2)
            end
        end
    end
    
    # real power min limit
    for g in G
        @NLconstraint(nlmod, pg_minus[g] - pg[g] <= 0) 
    end

    #real power max limit
    for g in G
        @NLconstraint(nlmod, pg[g] <= pg_plus[g])
    end

    #reactive power min limit
    for g in G
        @NLconstraint(nlmod, qg_minus[g] - qg[g] <= 0)
    end

    #reactive power max limit
    for g in G
        @NLconstraint(nlmod, qg[g] <= qg_plus[g])
    end
    
    #voltage level min limit
    for b in B
        @NLconstraint(nlmod, vb_minus[b] - vb[b] <= 0)
    end

    #voltage level max limit
    for b in B
        @NLconstraint(nlmod, vb[b] <= vb_plus[b])
    end

    #slack bus voltage phase 
    for i in 1:length(bus_types)
        if bus_types[i] == 3
            @NLconstraint(nlmod, δb[i] == 0)
        end
    end
    
    pp = NLPEvaluator(nlmod)
    MOI.initialize(pp, [:Jac,:Grad,:Hess])

    while !is_s_eq_0
        
        pp = NLPEvaluator(nlmod)
        MOI.initialize(pp, [:Jac,:Grad,:Hess])
        
        vals = [pk; qk; vk; δk; pbbfk; pbbtk; qbbfk; qbbtk]

        #Constraint eval
        gg = zeros(length(all_nonlinear_constraints(nlmod)))
        MOI.eval_constraint(pp,gg,vals)

        #Jacobian eval
        JStr=MOI.jacobian_structure(pp)
        J=zeros(length(JStr))
        MOI.eval_constraint_jacobian(pp,J,vals)

        jr = Vector{Int64}(undef, length(J)); jc = Vector{Int64}(undef, length(J))
        for i in 1:length(JStr)
            jr[i] = JStr[i][1] |> Int
            jc[i] = JStr[i][2] |> Int
        end
        Jacobeval = sparse(jr, jc, J) 
        Jacobi = Matrix(Jacobeval)
        
        
         m = Model(Ipopt.Optimizer)
        set_string_names_on_creation(m, false)
        set_silent(m)
        @variable(m, d_pg[g in G])
        @variable(m, d_qg[g in G])
        @variable(m, d_vb[b in B])
        @variable(m, d_δb[b in B])
        @variable(m, d_pbbf[b1 in B, b2 in E_b_f[b1]])
        @variable(m, d_pbbt[b1 in B, b2 in E_b_f[b1]])
        @variable(m, d_qbbf[b1 in B, b2 in E_b_f[b1]])
        @variable(m, d_qbbt[b1 in B, b2 in E_b_f[b1]])
        @variable(m, s[j=1:length(all_nonlinear_constraints(nlmod))] >= 0)
        
        @objective(m, Min, sum(s)) 

        
        #kcl for real power:
        for b in B
            @constraint(m, gg[b] + dot(Jacobi[b, 1:lg],d_pg) +
                                   dot(Jacobi[b, lg+1:2lg],d_qg) +
                                   dot(Jacobi[b, 2lg+1:(2lg+lb)], d_vb) +
                                   dot(Jacobi[b, (2lg+lb+1):(2lg+2lb)],d_δb) +
                                   dot(Jacobi[b, (2lg+2lb+1):(2lg+2lb+ll)], d_pbbf) + 
                                   dot(Jacobi[b, (2lg+2lb+ll+1):(2lg+2lb+2ll)], d_pbbt) +
                                   dot(Jacobi[b, (2lg+2lb+2ll+1):(2lg+2lb+3ll)], d_qbbf) +
                                   dot(Jacobi[b, (2lg+2lb+3ll+1):(2lg+2lb+4ll)], d_qbbt) - s[b] == 0)
        end
        

        #kcl for reactive power
        for b in (lb+1):2lb
            @constraint(m, gg[b] + dot(Jacobi[b, 1:lg],d_pg) +
                                   dot(Jacobi[b, lg+1:2lg],d_qg) +
                                   dot(Jacobi[b, 2lg+1:(2lg+lb)], d_vb) +
                                   dot(Jacobi[b, (2lg+lb+1):(2lg+2lb)],d_δb) +
                                   dot(Jacobi[b, (2lg+2lb+1):(2lg+2lb+ll)], d_pbbf) + 
                                   dot(Jacobi[b, (2lg+2lb+ll+1):(2lg+2lb+2ll)], d_pbbt) +
                                   dot(Jacobi[b, (2lg+2lb+2ll+1):(2lg+2lb+3ll)], d_qbbf) +
                                   dot(Jacobi[b, (2lg+2lb+3ll+1):(2lg+2lb+4ll)], d_qbbt) - s[b] == 0)
        end


        #kvl real power from
        for l in (2lb+1):(2lb+ll)
            @constraint(m, gg[l] + dot(Jacobi[l, 1:lg],d_pg) +
                                   dot(Jacobi[l, lg+1:2lg],d_qg) +
                                   dot(Jacobi[l, 2lg+1:(2lg+lb)], d_vb) +
                                   dot(Jacobi[l, (2lg+lb+1):(2lg+2lb)],d_δb) +
                                   dot(Jacobi[l, (2lg+2lb+1):(2lg+2lb+ll)], d_pbbf) + 
                                   dot(Jacobi[l, (2lg+2lb+ll+1):(2lg+2lb+2ll)], d_pbbt) +
                                   dot(Jacobi[l, (2lg+2lb+2ll+1):(2lg+2lb+3ll)], d_qbbf) +
                                   dot(Jacobi[l, (2lg+2lb+3ll+1):(2lg+2lb+4ll)], d_qbbt) - s[l] == 0)
        end
                
        #kvl real power to
        for l in (2lb+ll+1):(2lb+2ll)
            @constraint(m, gg[l] + dot(Jacobi[l, 1:lg],d_pg) +
                                   dot(Jacobi[l, lg+1:2lg],d_qg) +
                                   dot(Jacobi[l, 2lg+1:(2lg+lb)], d_vb) +
                                   dot(Jacobi[l, (2lg+lb+1):(2lg+2lb)],d_δb) +
                                   dot(Jacobi[l, (2lg+2lb+1):(2lg+2lb+ll)], d_pbbf) + 
                                   dot(Jacobi[l, (2lg+2lb+ll+1):(2lg+2lb+2ll)], d_pbbt) +
                                   dot(Jacobi[l, (2lg+2lb+2ll+1):(2lg+2lb+3ll)], d_qbbf) +
                                   dot(Jacobi[l, (2lg+2lb+3ll+1):(2lg+2lb+4ll)], d_qbbt) - s[l] == 0)
        end
        
        #kvl reactive power from
        for l in (2lb+2ll+1):(2lb+3ll)
            @constraint(m, gg[l] + dot(Jacobi[l, 1:lg],d_pg) +
                                   dot(Jacobi[l, lg+1:2lg],d_qg) +
                                   dot(Jacobi[l, 2lg+1:(2lg+lb)], d_vb) +
                                   dot(Jacobi[l, (2lg+lb+1):(2lg+2lb)],d_δb) +
                                   dot(Jacobi[l, (2lg+2lb+1):(2lg+2lb+ll)], d_pbbf) + 
                                   dot(Jacobi[l, (2lg+2lb+ll+1):(2lg+2lb+2ll)], d_pbbt) +
                                   dot(Jacobi[l, (2lg+2lb+2ll+1):(2lg+2lb+3ll)], d_qbbf) +
                                   dot(Jacobi[l, (2lg+2lb+3ll+1):(2lg+2lb+4ll)], d_qbbt) - s[l] == 0)
        end
        
        #kvl reactive power to
        for l in (2lb+3ll+1):(2lb+4ll)
            @constraint(m, gg[l] + dot(Jacobi[l, 1:lg],d_pg) +
                                   dot(Jacobi[l, lg+1:2lg],d_qg) +
                                   dot(Jacobi[l, 2lg+1:(2lg+lb)], d_vb) +
                                   dot(Jacobi[l, (2lg+lb+1):(2lg+2lb)],d_δb) +
                                   dot(Jacobi[l, (2lg+2lb+1):(2lg+2lb+ll)], d_pbbf) + 
                                   dot(Jacobi[l, (2lg+2lb+ll+1):(2lg+2lb+2ll)], d_pbbt) +
                                   dot(Jacobi[l, (2lg+2lb+2ll+1):(2lg+2lb+3ll)], d_qbbf) +
                                   dot(Jacobi[l, (2lg+2lb+3ll+1):(2lg+2lb+4ll)], d_qbbt) - s[l] == 0)
        end       
       
        #thermal limits from
        for l in (2lb+4ll+1):(2lb+4ll+therm_lim_ctr)
            @constraint(m, gg[l] + dot(Jacobi[l, 1:lg],d_pg) +
                                   dot(Jacobi[l, lg+1:2lg],d_qg) +
                                   dot(Jacobi[l, 2lg+1:(2lg+lb)], d_vb) +
                                   dot(Jacobi[l, (2lg+lb+1):(2lg+2lb)],d_δb) +
                                   dot(Jacobi[l, (2lg+2lb+1):(2lg+2lb+ll)], d_pbbf) + 
                                   dot(Jacobi[l, (2lg+2lb+ll+1):(2lg+2lb+2ll)], d_pbbt) +
                                   dot(Jacobi[l, (2lg+2lb+2ll+1):(2lg+2lb+3ll)], d_qbbf) +
                                   dot(Jacobi[l, (2lg+2lb+3ll+1):(2lg+2lb+4ll)], d_qbbt) - s[l] <= 0)
        end
        
        #thermal limits to
        for l in (2lb+4ll+1+therm_lim_ctr):(2lb+4ll+2therm_lim_ctr)
            @constraint(m, gg[l] + dot(Jacobi[l, 1:lg],d_pg) +
                                   dot(Jacobi[l, lg+1:2lg],d_qg) +
                                   dot(Jacobi[l, 2lg+1:(2lg+lb)], d_vb) +
                                   dot(Jacobi[l, (2lg+lb+1):(2lg+2lb)],d_δb) +
                                   dot(Jacobi[l, (2lg+2lb+1):(2lg+2lb+ll)], d_pbbf) + 
                                   dot(Jacobi[l, (2lg+2lb+ll+1):(2lg+2lb+2ll)], d_pbbt) +
                                   dot(Jacobi[l, (2lg+2lb+2ll+1):(2lg+2lb+3ll)], d_qbbf) +
                                   dot(Jacobi[l, (2lg+2lb+3ll+1):(2lg+2lb+4ll)], d_qbbt) - s[l] <= 0)
        end
        
        # real power min limit
        for g in (2lb+4ll+1+2therm_lim_ctr):(2lb+4ll+2therm_lim_ctr+lg)
            @constraint(m, gg[g] + dot(Jacobi[g, 1:lg],d_pg) +
                                   dot(Jacobi[g, lg+1:2lg],d_qg) +
                                   dot(Jacobi[g, 2lg+1:(2lg+lb)], d_vb) +
                                   dot(Jacobi[g, (2lg+lb+1):(2lg+2lb)],d_δb) +
                                   dot(Jacobi[g, (2lg+2lb+1):(2lg+2lb+ll)], d_pbbf) + 
                                   dot(Jacobi[g, (2lg+2lb+ll+1):(2lg+2lb+2ll)], d_pbbt) +
                                   dot(Jacobi[g, (2lg+2lb+2ll+1):(2lg+2lb+3ll)], d_qbbf) +
                                   dot(Jacobi[g, (2lg+2lb+3ll+1):(2lg+2lb+4ll)], d_qbbt) - s[g] <= 0) 
        end

        #real power max limit
        for g in (2lb+4ll+1+2therm_lim_ctr+lg):(2lb+4ll+2therm_lim_ctr+2lg)
            @constraint(m, gg[g] + dot(Jacobi[g, 1:lg],d_pg) +
                                   dot(Jacobi[g, lg+1:2lg],d_qg) +
                                   dot(Jacobi[g, 2lg+1:(2lg+lb)], d_vb) +
                                   dot(Jacobi[g, (2lg+lb+1):(2lg+2lb)],d_δb) +
                                   dot(Jacobi[g, (2lg+2lb+1):(2lg+2lb+ll)], d_pbbf) + 
                                   dot(Jacobi[g, (2lg+2lb+ll+1):(2lg+2lb+2ll)], d_pbbt) +
                                   dot(Jacobi[g, (2lg+2lb+2ll+1):(2lg+2lb+3ll)], d_qbbf) +
                                   dot(Jacobi[g, (2lg+2lb+3ll+1):(2lg+2lb+4ll)], d_qbbt) - s[g] <= 0)
        end

        #reactive power min limit
        for g in (2lb+4ll+1+2therm_lim_ctr+2lg):(2lb+4ll+2therm_lim_ctr+3lg)
            @constraint(m, gg[g] + dot(Jacobi[g, 1:lg],d_pg) +
                                   dot(Jacobi[g, lg+1:2lg],d_qg) +
                                   dot(Jacobi[g, 2lg+1:(2lg+lb)], d_vb) +
                                   dot(Jacobi[g, (2lg+lb+1):(2lg+2lb)],d_δb) +
                                   dot(Jacobi[g, (2lg+2lb+1):(2lg+2lb+ll)], d_pbbf) + 
                                   dot(Jacobi[g, (2lg+2lb+ll+1):(2lg+2lb+2ll)], d_pbbt) +
                                   dot(Jacobi[g, (2lg+2lb+2ll+1):(2lg+2lb+3ll)], d_qbbf) +
                                   dot(Jacobi[g, (2lg+2lb+3ll+1):(2lg+2lb+4ll)], d_qbbt) - s[g] <= 0)
        end
        
        #reactive power max limit
        for g in (2lb+4ll+1+2therm_lim_ctr+3lg):(2lb+4ll+2therm_lim_ctr+4lg)
            @constraint(m, gg[g] + dot(Jacobi[g, 1:lg],d_pg) +
                                   dot(Jacobi[g, lg+1:2lg],d_qg) +
                                   dot(Jacobi[g, 2lg+1:(2lg+lb)], d_vb) +
                                   dot(Jacobi[g, (2lg+lb+1):(2lg+2lb)],d_δb) +
                                   dot(Jacobi[g, (2lg+2lb+1):(2lg+2lb+ll)], d_pbbf) + 
                                   dot(Jacobi[g, (2lg+2lb+ll+1):(2lg+2lb+2ll)], d_pbbt) +
                                   dot(Jacobi[g, (2lg+2lb+2ll+1):(2lg+2lb+3ll)], d_qbbf) +
                                   dot(Jacobi[g, (2lg+2lb+3ll+1):(2lg+2lb+4ll)], d_qbbt) - s[g] <= 0)
        end
        
        #voltage level min limit
        for b in (2lb+4ll+1+2therm_lim_ctr+4lg):(3lb+4ll+2therm_lim_ctr+4lg)
            @constraint(m, gg[b] + dot(Jacobi[b, 1:lg],d_pg) +
                                   dot(Jacobi[b, lg+1:2lg],d_qg) +
                                   dot(Jacobi[b, 2lg+1:(2lg+lb)], d_vb) +
                                   dot(Jacobi[b, (2lg+lb+1):(2lg+2lb)],d_δb) +
                                   dot(Jacobi[b, (2lg+2lb+1):(2lg+2lb+ll)], d_pbbf) + 
                                   dot(Jacobi[b, (2lg+2lb+ll+1):(2lg+2lb+2ll)], d_pbbt) +
                                   dot(Jacobi[b, (2lg+2lb+2ll+1):(2lg+2lb+3ll)], d_qbbf) +
                                   dot(Jacobi[b, (2lg+2lb+3ll+1):(2lg+2lb+4ll)], d_qbbt) - s[b] <= 0)
        end
        
        #voltage level max limit
        for b in (3lb+4ll+1+2therm_lim_ctr+4lg):(4lb+4ll+2therm_lim_ctr+4lg)
            @constraint(m, gg[b] + dot(Jacobi[b, 1:lg],d_pg) +
                                   dot(Jacobi[b, lg+1:2lg],d_qg) +
                                   dot(Jacobi[b, 2lg+1:(2lg+lb)], d_vb) +
                                   dot(Jacobi[b, (2lg+lb+1):(2lg+2lb)],d_δb) +
                                   dot(Jacobi[b, (2lg+2lb+1):(2lg+2lb+ll)], d_pbbf) + 
                                   dot(Jacobi[b, (2lg+2lb+ll+1):(2lg+2lb+2ll)], d_pbbt) +
                                   dot(Jacobi[b, (2lg+2lb+2ll+1):(2lg+2lb+3ll)], d_qbbf) +
                                   dot(Jacobi[b, (2lg+2lb+3ll+1):(2lg+2lb+4ll)], d_qbbt) - s[b] <= 0)
        end

        #slack bus voltage phase 
        for r = (4lb+4ll+1+2therm_lim_ctr+4lg)
            @constraint(m, gg[r] + dot(Jacobi[r, 1:lg],d_pg) +
                                   dot(Jacobi[r, lg+1:2lg],d_qg) +
                                   dot(Jacobi[r, 2lg+1:(2lg+lb)], d_vb) +
                                   dot(Jacobi[r, (2lg+lb+1):(2lg+2lb)],d_δb) +
                                   dot(Jacobi[r, (2lg+2lb+1):(2lg+2lb+ll)], d_pbbf) + 
                                   dot(Jacobi[r, (2lg+2lb+ll+1):(2lg+2lb+2ll)], d_pbbt) +
                                   dot(Jacobi[r, (2lg+2lb+2ll+1):(2lg+2lb+3ll)], d_qbbf) +
                                   dot(Jacobi[r, (2lg+2lb+3ll+1):(2lg+2lb+4ll)], d_qbbt) - s[r] == 0)
        end
        
        @assert size(Jacobi, 1) == (4lb+4ll+1+2therm_lim_ctr+4lg)
     
        # trust region constraints        
        @constraint(m, d_pg .<= ρ)
        @constraint(m, d_qg .<= ρ)
        @constraint(m, d_vb .<= ρ)
        @constraint(m, d_δb .<= ρ)
        @constraint(m, d_pbbf .<= ρ)
        @constraint(m, d_pbbt .<= ρ)
        @constraint(m, d_qbbf .<= ρ)
        @constraint(m, d_qbbt .<= ρ)
        @constraint(m, d_pg .>= -ρ)
        @constraint(m, d_qg .>= -ρ)
        @constraint(m, d_vb .>= -ρ)
        @constraint(m, d_δb .>= -ρ)
        @constraint(m, d_pbbf .>= -ρ)
        @constraint(m, d_pbbt .>= -ρ)
        @constraint(m, d_qbbf .>= -ρ)
        @constraint(m, d_pbbt .>= -ρ)
  
        optimize!(m)
        
        # updating difference values
        prop_d_pg = pk .+ value.(d_pg)
        prop_d_qg = qk .+ value.(d_qg)
        prop_d_vb = vk .+ value.(d_vb)
        prop_d_δb = δk .+ value.(d_δb)      
        
        prop_d_pbbf = []
        prop_d_qbbf = []
        prop_d_pbbt = []
        prop_d_qbbt = []
        ctr = 0
        for b1 in B
            for b2 in E_b_f[b1]
                ctr +=1
                prop_d_pbbf = push!(prop_d_pbbf, pbbfk[ctr] + value.(d_pbbf[b1, b2]))
                prop_d_qbbf = push!(prop_d_qbbf, qbbfk[ctr] + value.(d_qbbf[b1, b2]))
                prop_d_pbbt = push!(prop_d_pbbt, pbbtk[ctr] + value.(d_pbbt[b1, b2]))
                prop_d_qbbt = push!(prop_d_qbbt, qbbtk[ctr] + value.(d_qbbt[b1, b2]))
            end
        end
    
        
        # convert all new points into vectors of float64 types
        prop_d_pg = convert(Vector{Float64},prop_d_pg)
        prop_d_qg = convert(Vector{Float64},prop_d_qg)
        prop_d_vb = convert(Vector{Float64},prop_d_vb)
        prop_d_δb = convert(Vector{Float64},prop_d_δb)
        prop_d_pbbf = convert(Vector{Float64},prop_d_pbbf)
        prop_d_pbbt = convert(Vector{Float64},prop_d_pbbt)
        prop_d_qbbf = convert(Vector{Float64},prop_d_qbbf)
        prop_d_qbbt = convert(Vector{Float64},prop_d_qbbt)

        prop_vals = [prop_d_pg;prop_d_qg;prop_d_vb;prop_d_δb;prop_d_pbbf;prop_d_pbbt;prop_d_qbbf;prop_d_qbbt]
        
        # merit function
        function mer(x::Vector) 
            ce = zeros(length(all_nonlinear_constraints(nlmod)))
            MOI.eval_constraint(pp,ce,x)
            
            eq_rows = ce[1:(2lb+4ll)]
            eq_rows = vcat(eq_rows, ce[end])
            ineq_rows = ce[(2lb+4ll+1):end-1]
            @assert length([eq_rows;ineq_rows])==length(ce)
            
            max_ineq_rows = []
            for i in 1:length(ineq_rows)
                maxval = max(ineq_rows[i], 0)
                max_ineq_rows = push!(max_ineq_rows, maxval)
            end
            
            merit = norm(eq_rows) + norm(max_ineq_rows)
            return merit
        end
        
        diff_vars = vcat(value.(d_pg), value.(d_qg), value.(d_vb), value.(d_δb), [value.(d_pbbf[b1,b2]) for b1 in B for b2 in E_b_f[b1]], [value.(d_pbbt[b1,b2]) for b1 in B for b2 in E_b_f[b1]], [value.(d_qbbf[b1,b2]) for b1 in B for b2 in E_b_f[b1]], [value.(d_qbbt[b1,b2]) for b1 in B for b2 in E_b_f[b1]])
        
        if all(v -> abs(v) < 1e-6, value.(s))
        elseif mer(prop_vals) < mer(vals) 
            println("restoration merit condition: success")
            pk =prop_d_pg
            qk =prop_d_qg
            vk = prop_d_vb
            δk =prop_d_δb
            pbbfk = prop_d_pbbf
            pbbtk = prop_d_pbbt
            qbbfk = prop_d_qbbf
            qbbtk = prop_d_qbbt
            ρ += 1*ρ
        else
            println("restoration merit condition: failed")
#             ρ -= 0.5*ρ
            max_step = norm(diff_vars, Inf) # more aggressive trust region
            ρ = max_step/3
        end
                
        is_s_eq_0 = sum(value.(s)) < 1e-2

    end
    
    return pk,qk,vk,δk,pbbfk,pbbtk,qbbfk,qbbtk, ρ
end

restoration_phase (generic function with 1 method)

# Nonlinear solver

In [7]:
function OPF_NL()
    
    nlmod = Model(Ipopt.Optimizer)
    set_string_names_on_creation(nlmod, false)
#     set_silent(nlmod)
    
    @variable(nlmod, pg[g in G]) # real power generation
    @variable(nlmod, qg[g in G]) # reactive power generation
    @variable(nlmod, vb[b in B],(start=1.0)) # bus voltage
    @variable(nlmod, δb[b in B],(start=0.0)) # bus voltage phase
    @variable(nlmod, pbbf[b1 in B, b2 in E_b_f[b1]]) # line real power (from)
    @variable(nlmod, pbbt[b1 in B, b2 in E_b_f[b1]]) # line real power (to)
    @variable(nlmod, qbbf[b1 in B, b2 in E_b_f[b1]]) # line reactive power (from)
    @variable(nlmod, qbbt[b1 in B, b2 in E_b_f[b1]]) # line reactive power (to)

    
    @NLobjective(nlmod, Min, sum(100^2*gen_c[g][1] * pg[g]^2 + 100*gen_c[g][2] * pg[g] + gen_c[g][3] for g in G))
    
    # kcl for real power
    for b in B
        @NLconstraint(nlmod, sum(pg[g] for g in β_g[b]) == demand_real[b] +
                                                           sum(pbbf[b,b_] for b_ in E_b_f[b]) +
                                                           sum(pbbt[b_,b] for b_ in E_b_t[b]) +
                                                           shunt_conductance[b] * (vb[b])^2)
    end
    
    # kcl for reactive power
    for b in B
        @NLconstraint(nlmod, sum(qg[g] for g in β_g[b]) == demand_reactive[b] + sum(qbbf[b,b_] for b_ in E_b_f[b]) +
                                sum(qbbt[b_,b] for b_ in E_b_t[b]) - shunt_susceptance[b] * (vb[b])^2)
    end
    
# kvl real power from
    for b1 in B
        for b2 in E_b_f[b1]
            @NLconstraint(nlmod, line_con[(b1,b2)]* (vb[b1])^2 - (vb[b1])*(vb[b2])*
                    (line_con[(b1,b2)] * cos(δb[b1] - δb[b2]) + line_sus[(b1,b2)] * sin(δb[b1] - δb[b2])) == pbbf[b1, b2])
        end
    end
    
    # kvl real power to
    for b1 in B
        for b2 in E_b_f[b1]
            @NLconstraint(nlmod, line_con[(b2,b1)]* (vb[b2])^2 - (vb[b2])*(vb[b1])*
                    (line_con[(b2,b1)] * cos(δb[b2] - δb[b1]) + line_sus[(b2,b1)] * sin(δb[b2] - δb[b1])) == pbbt[b1, b2])
        end
    end
    
    # kvl reactive power from
    for b1 in B
        for b2 in E_b_f[b1]
            @NLconstraint(nlmod, -line_sus[(b1,b2)]* (vb[b1])^2 - (vb[b1])*(vb[b2])*
                        (line_con[(b1,b2)] * sin(δb[b1] - δb[b2]) - line_sus[(b1,b2)] * cos(δb[b1] - δb[b2])) == qbbf[b1, b2])
        end
    end
        
    # kvl reactive power to
    for b1 in B
        for b2 in E_b_f[b1]
            @NLconstraint(nlmod, -line_sus[(b2,b1)]* (vb[b2])^2 - (vb[b2])*(vb[b1])*
                        (line_con[(b2,b1)] * sin(δb[b2] - δb[b1]) - line_sus[(b2,b1)] * cos(δb[b2] - δb[b1])) == qbbt[b1, b2])
        end
    end
    
    # thermal limits from
    for b1 in B
        for b2 in E_b_f[b1]
            if line_therm_lim[(b1, b2)] != 0.0
                @NLconstraint(nlmod, (pbbf[b1, b2])^2 + (qbbf[b1, b2])^2 <= (0.01*line_therm_lim[(b1, b2)])^2)
            end
        end   
    end
    
    # thermal limits to
    for b1 in B
        for b2 in E_b_f[b1]
            if line_therm_lim[(b1, b2)] != 0.0
                @NLconstraint(nlmod, (pbbt[b1, b2])^2 + (qbbt[b1, b2])^2 <= (0.01*line_therm_lim[(b1, b2)])^2)
            end
        end
    end
    
    # real power min limit
    for g in G
        @NLconstraint(nlmod, pg_minus[g] - pg[g] <= 0) 
    end

    #real power max limit
    for g in G
        @NLconstraint(nlmod, pg[g] <= pg_plus[g])
    end

    #reactive power min limit
    for g in G
        @NLconstraint(nlmod, qg_minus[g] - qg[g] <= 0)
    end

    #reactive power max limit
    for g in G
        @NLconstraint(nlmod, qg[g] <= qg_plus[g])
    end
    
    #voltage level min limit
    for b in B
        @NLconstraint(nlmod, vb_minus[b] - vb[b] <= 0)
    end

    #voltage level max limit
    for b in B
        @NLconstraint(nlmod, vb[b] <= vb_plus[b])
    end

    #slack bus voltage phase 
    for i in 1:length(bus_types)
        if bus_types[i] == 3
            @NLconstraint(nlmod, δb[i] == 0)
        end
    end
    
    optimize!(nlmod)

    @show objective_value(nlmod)
    @show value.(pg)  
    @show value.(qg)
    @show value.(vb)
    @show value.(δb)
    @show value.(pbbf)
    @show value.(pbbt)
    @show value.(qbbf)
    @show value.(qbbt)
    
end

OPF_NL (generic function with 1 method)

In [8]:
OPF_NL()


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:      241
Number of nonzeros in inequality constraint Jacobian.:       66
Number of nonzeros in Lagrangian Hessian.............:      417

Total number of variables............................:       60
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:       55
Total number of inequality co

JuMP.Containers.SparseAxisArray{Float64, 2, Tuple{Int64, Int64}} with 9 entries:
  [1, 4]  =  -0.630589
  [3, 6]  =  -0.345786
  [4, 5]  =  -0.205617
  [5, 6]  =  0.143291
  [6, 7]  =  -0.186354
  [7, 8]  =  0.190127
  [8, 2]  =  0.49885
  [8, 9]  =  -0.12424
  [9, 4]  =  0.410976

# SLP with a merit function

In [9]:
function SLP_MF_OPF(pk, qk, vk, δk, pbbfk, pbbtk, qbbfk, qbbtk, ρ, γ; maxiter=100)

    is_converged = false
    
    # nonlinear model for moi evaluation
    nlmod = Model()
    set_string_names_on_creation(nlmod, false)
#     set_silent(nlmod)
    
    @variable(nlmod, pg[g in G]) # real power generation
    @variable(nlmod, qg[g in G]) # reactive power generation
    @variable(nlmod, vb[b in B],(start=1.0)) # bus voltage
    @variable(nlmod, δb[b in B],(start=0.0)) # bus voltage phase
    @variable(nlmod, pbbf[b1 in B, b2 in E_b_f[b1]]) # line real power (from)
    @variable(nlmod, pbbt[b1 in B, b2 in E_b_f[b1]]) # line real power (to)
    @variable(nlmod, qbbf[b1 in B, b2 in E_b_f[b1]]) # line reactive power (from)
    @variable(nlmod, qbbt[b1 in B, b2 in E_b_f[b1]]) # line reactive power (to)

    
    @NLobjective(nlmod, Min, sum(100^2*gen_c[g][1] * pg[g]^2 + 100*gen_c[g][2] * pg[g] + gen_c[g][3] for g in G))
    
    # kcl for real power
    for b in B
        @NLconstraint(nlmod, sum(pg[g] for g in β_g[b]) == demand_real[b] +
                                                           sum(pbbf[b,b_] for b_ in E_b_f[b]) +
                                                           sum(pbbt[b_,b] for b_ in E_b_t[b]) +
                                                           shunt_conductance[b] * (vb[b])^2)
    end
    
    # kcl for reactive power
    for b in B
        @NLconstraint(nlmod, sum(qg[g] for g in β_g[b]) == demand_reactive[b] + sum(qbbf[b,b_] for b_ in E_b_f[b]) +
                                sum(qbbt[b_,b] for b_ in E_b_t[b]) - shunt_susceptance[b] * (vb[b])^2)
    end
    
    # kvl real power from
    for b1 in B
        for b2 in E_b_f[b1]
            @NLconstraint(nlmod, line_con[(b1,b2)]* (vb[b1])^2 - vb[b1]*vb[b2]*
                    (line_con[(b1,b2)] * cos(δb[b1] - δb[b2]) + line_sus[(b1,b2)] * sin(δb[b1] - δb[b2])) == pbbf[b1, b2])
        end
    end
    
    # kvl real power to
    for b1 in B
        for b2 in E_b_f[b1]
            @NLconstraint(nlmod, line_con[(b2,b1)]* (vb[b2])^2 - vb[b2]*vb[b1]*
                    (line_con[(b2,b1)] * cos(δb[b2] - δb[b1]) + line_sus[(b2,b1)] * sin(δb[b2] - δb[b1])) == pbbt[b1, b2])
        end
    end
    
    # kvl reactive power from
    for b1 in B
        for b2 in E_b_f[b1]
            @NLconstraint(nlmod, -line_sus[(b1,b2)]* (vb[b1])^2 - vb[b1]*vb[b2]*
                        (line_con[(b1,b2)] * sin(δb[b1] - δb[b2]) - line_sus[(b1,b2)] * cos(δb[b1] - δb[b2])) == qbbf[b1, b2])
        end
    end
        
    # kvl reactive power to
    for b1 in B
        for b2 in E_b_f[b1]
            @NLconstraint(nlmod, -line_sus[(b2,b1)]* (vb[b2])^2 - vb[b2]*vb[b1]*
                        (line_con[(b2,b1)] * sin(δb[b2] - δb[b1]) - line_sus[(b2,b1)] * cos(δb[b2] - δb[b1])) == qbbt[b1, b2])
        end
    end
    
    # thermal limits from
    therm_lim_ctr = 0
    for b1 in B
        for b2 in E_b_f[b1]
            if line_therm_lim[(b1, b2)] != 0.0
                @NLconstraint(nlmod, (pbbf[b1, b2])^2 + (qbbf[b1, b2])^2 <= (0.01*line_therm_lim[(b1, b2)])^2)
                therm_lim_ctr += 1
            end
        end   
    end
    
    # thermal limits to
    for b1 in B
        for b2 in E_b_f[b1]
            if line_therm_lim[(b1, b2)] != 0.0
                @NLconstraint(nlmod, (pbbt[b1, b2])^2 + (qbbt[b1, b2])^2 <= (0.01*line_therm_lim[(b1, b2)])^2)
            end
        end
    end
    
    # real power min limit
    for g in G
        @NLconstraint(nlmod, pg_minus[g] - pg[g] <= 0) 
    end

    #real power max limit
    for g in G
        @NLconstraint(nlmod, pg[g] <= pg_plus[g])
    end

    #reactive power min limit
    for g in G
        @NLconstraint(nlmod, qg_minus[g] - qg[g] <= 0)
    end

    #reactive power max limit
    for g in G
        @NLconstraint(nlmod, qg[g] <= qg_plus[g])
    end
    
    #voltage level min limit
    for b in B
        @NLconstraint(nlmod, vb_minus[b] - vb[b] <= 0)
    end

    #voltage level max limit
    for b in B
        @NLconstraint(nlmod, vb[b] <= vb_plus[b])
    end

    #slack bus voltage phase 
    for i in 1:length(bus_types)
        if bus_types[i] == 3
            @NLconstraint(nlmod, δb[i] == 0)
        end
    end

    d = NLPEvaluator(nlmod)
    MOI.initialize(d, [:Jac,:Grad,:Hess])
    
    
    for i in 1:maxiter 
        if is_converged
            break
        end
        
        d = NLPEvaluator(nlmod)
        MOI.initialize(d, [:Jac,:Grad,:Hess])
        
        vals = [pk; qk; vk; δk; pbbfk; pbbtk; qbbfk; qbbtk]
        #Objective eval
        objEv=MOI.eval_objective(d, pk) 

        #Objective gradient
        df=zeros(length(pk))
        MOI.eval_objective_gradient(d,df,pk)

        #Constraint eval
        gg = zeros(length(all_nonlinear_constraints(nlmod)))
        MOI.eval_constraint(d,gg,vals)

        #Jacobian eval
        JStr=MOI.jacobian_structure(d)
        J=zeros(length(JStr))
        MOI.eval_constraint_jacobian(d,J,vals)

        jr = Vector{Int64}(undef, length(J)); jc = Vector{Int64}(undef, length(J))
        for i in 1:length(JStr)
            jr[i] = JStr[i][1] |> Int
            jc[i] = JStr[i][2] |> Int
        end
        Jacobeval = sparse(jr, jc, J) 
        Jacobi = Matrix(Jacobeval)
        
            
        # linear model    
        m = Model(Ipopt.Optimizer)
        set_optimizer_attribute(m, "tol", 10^(-10))
#         m = Model(Gurobi.Optimizer)
        set_string_names_on_creation(m, false)
#         set_silent(m)
        @variable(m, d_pg[g in G])
        @variable(m, d_qg[g in G])
        @variable(m, d_vb[b in B])
        @variable(m, d_δb[b in B])
        @variable(m, d_pbbf[b1 in B, b2 in E_b_f[b1]])
        @variable(m, d_pbbt[b1 in B, b2 in E_b_f[b1]])
        @variable(m, d_qbbf[b1 in B, b2 in E_b_f[b1]])
        @variable(m, d_qbbt[b1 in B, b2 in E_b_f[b1]])
        
        @objective(m, Min, objEv + df'*d_pg) 

        
        #kcl for real power:
        for b in B
            @constraint(m, gg[b] + dot(Jacobi[b, 1:lg],d_pg) +
                                   dot(Jacobi[b, lg+1:2lg],d_qg) +
                                   dot(Jacobi[b, 2lg+1:(2lg+lb)], d_vb) +
                                   dot(Jacobi[b, (2lg+lb+1):(2lg+2lb)],d_δb) +
                                   dot(Jacobi[b, (2lg+2lb+1):(2lg+2lb+ll)], d_pbbf) + 
                                   dot(Jacobi[b, (2lg+2lb+ll+1):(2lg+2lb+2ll)], d_pbbt) +
                                   dot(Jacobi[b, (2lg+2lb+2ll+1):(2lg+2lb+3ll)], d_qbbf) +
                                   dot(Jacobi[b, (2lg+2lb+3ll+1):(2lg+2lb+4ll)], d_qbbt) == 0)
        end
        

        #kcl for reactive power
        for b in (lb+1):2lb
            @constraint(m, gg[b] + dot(Jacobi[b, 1:lg],d_pg) +
                                   dot(Jacobi[b, lg+1:2lg],d_qg) +
                                   dot(Jacobi[b, 2lg+1:(2lg+lb)], d_vb) +
                                   dot(Jacobi[b, (2lg+lb+1):(2lg+2lb)],d_δb) +
                                   dot(Jacobi[b, (2lg+2lb+1):(2lg+2lb+ll)], d_pbbf) + 
                                   dot(Jacobi[b, (2lg+2lb+ll+1):(2lg+2lb+2ll)], d_pbbt) +
                                   dot(Jacobi[b, (2lg+2lb+2ll+1):(2lg+2lb+3ll)], d_qbbf) +
                                   dot(Jacobi[b, (2lg+2lb+3ll+1):(2lg+2lb+4ll)], d_qbbt) == 0)
        end


        #kvl real power from
        for l in (2lb+1):(2lb+ll)
            @constraint(m, gg[l] + dot(Jacobi[l, 1:lg],d_pg) +
                                   dot(Jacobi[l, lg+1:2lg],d_qg) +
                                   dot(Jacobi[l, 2lg+1:(2lg+lb)], d_vb) +
                                   dot(Jacobi[l, (2lg+lb+1):(2lg+2lb)],d_δb) +
                                   dot(Jacobi[l, (2lg+2lb+1):(2lg+2lb+ll)], d_pbbf) + 
                                   dot(Jacobi[l, (2lg+2lb+ll+1):(2lg+2lb+2ll)], d_pbbt) +
                                   dot(Jacobi[l, (2lg+2lb+2ll+1):(2lg+2lb+3ll)], d_qbbf) +
                                   dot(Jacobi[l, (2lg+2lb+3ll+1):(2lg+2lb+4ll)], d_qbbt) == 0)
        end
                
        #kvl real power to
        for l in (2lb+ll+1):(2lb+2ll)
            @constraint(m, gg[l] + dot(Jacobi[l, 1:lg],d_pg) +
                                   dot(Jacobi[l, lg+1:2lg],d_qg) +
                                   dot(Jacobi[l, 2lg+1:(2lg+lb)], d_vb) +
                                   dot(Jacobi[l, (2lg+lb+1):(2lg+2lb)],d_δb) +
                                   dot(Jacobi[l, (2lg+2lb+1):(2lg+2lb+ll)], d_pbbf) + 
                                   dot(Jacobi[l, (2lg+2lb+ll+1):(2lg+2lb+2ll)], d_pbbt) +
                                   dot(Jacobi[l, (2lg+2lb+2ll+1):(2lg+2lb+3ll)], d_qbbf) +
                                   dot(Jacobi[l, (2lg+2lb+3ll+1):(2lg+2lb+4ll)], d_qbbt) == 0)
        end
        
        #kvl reactive power from
        for l in (2lb+2ll+1):(2lb+3ll)
            @constraint(m, gg[l] + dot(Jacobi[l, 1:lg],d_pg) +
                                   dot(Jacobi[l, lg+1:2lg],d_qg) +
                                   dot(Jacobi[l, 2lg+1:(2lg+lb)], d_vb) +
                                   dot(Jacobi[l, (2lg+lb+1):(2lg+2lb)],d_δb) +
                                   dot(Jacobi[l, (2lg+2lb+1):(2lg+2lb+ll)], d_pbbf) + 
                                   dot(Jacobi[l, (2lg+2lb+ll+1):(2lg+2lb+2ll)], d_pbbt) +
                                   dot(Jacobi[l, (2lg+2lb+2ll+1):(2lg+2lb+3ll)], d_qbbf) +
                                   dot(Jacobi[l, (2lg+2lb+3ll+1):(2lg+2lb+4ll)], d_qbbt) == 0)
        end
        
        #kvl reactive power to
        for l in (2lb+3ll+1):(2lb+4ll)
            @constraint(m, gg[l] + dot(Jacobi[l, 1:lg],d_pg) +
                                   dot(Jacobi[l, lg+1:2lg],d_qg) +
                                   dot(Jacobi[l, 2lg+1:(2lg+lb)], d_vb) +
                                   dot(Jacobi[l, (2lg+lb+1):(2lg+2lb)],d_δb) +
                                   dot(Jacobi[l, (2lg+2lb+1):(2lg+2lb+ll)], d_pbbf) + 
                                   dot(Jacobi[l, (2lg+2lb+ll+1):(2lg+2lb+2ll)], d_pbbt) +
                                   dot(Jacobi[l, (2lg+2lb+2ll+1):(2lg+2lb+3ll)], d_qbbf) +
                                   dot(Jacobi[l, (2lg+2lb+3ll+1):(2lg+2lb+4ll)], d_qbbt) == 0)
        end       
       
        #thermal limits from
        for l in (2lb+4ll+1):(2lb+4ll+therm_lim_ctr)
            @constraint(m, gg[l] + dot(Jacobi[l, 1:lg],d_pg) +
                                   dot(Jacobi[l, lg+1:2lg],d_qg) +
                                   dot(Jacobi[l, 2lg+1:(2lg+lb)], d_vb) +
                                   dot(Jacobi[l, (2lg+lb+1):(2lg+2lb)],d_δb) +
                                   dot(Jacobi[l, (2lg+2lb+1):(2lg+2lb+ll)], d_pbbf) + 
                                   dot(Jacobi[l, (2lg+2lb+ll+1):(2lg+2lb+2ll)], d_pbbt) +
                                   dot(Jacobi[l, (2lg+2lb+2ll+1):(2lg+2lb+3ll)], d_qbbf) +
                                   dot(Jacobi[l, (2lg+2lb+3ll+1):(2lg+2lb+4ll)], d_qbbt) <= 0)
        end
        
        #thermal limits to
        for l in (2lb+4ll+1+therm_lim_ctr):(2lb+4ll+2therm_lim_ctr)
            @constraint(m, gg[l] + dot(Jacobi[l, 1:lg],d_pg) +
                                   dot(Jacobi[l, lg+1:2lg],d_qg) +
                                   dot(Jacobi[l, 2lg+1:(2lg+lb)], d_vb) +
                                   dot(Jacobi[l, (2lg+lb+1):(2lg+2lb)],d_δb) +
                                   dot(Jacobi[l, (2lg+2lb+1):(2lg+2lb+ll)], d_pbbf) + 
                                   dot(Jacobi[l, (2lg+2lb+ll+1):(2lg+2lb+2ll)], d_pbbt) +
                                   dot(Jacobi[l, (2lg+2lb+2ll+1):(2lg+2lb+3ll)], d_qbbf) +
                                   dot(Jacobi[l, (2lg+2lb+3ll+1):(2lg+2lb+4ll)], d_qbbt) <= 0)
        end
        
        # real power min limit
        for g in (2lb+4ll+1+2therm_lim_ctr):(2lb+4ll+2therm_lim_ctr+lg)
            @constraint(m, gg[g] + dot(Jacobi[g, 1:lg],d_pg) +
                                   dot(Jacobi[g, lg+1:2lg],d_qg) +
                                   dot(Jacobi[g, 2lg+1:(2lg+lb)], d_vb) +
                                   dot(Jacobi[g, (2lg+lb+1):(2lg+2lb)],d_δb) +
                                   dot(Jacobi[g, (2lg+2lb+1):(2lg+2lb+ll)], d_pbbf) + 
                                   dot(Jacobi[g, (2lg+2lb+ll+1):(2lg+2lb+2ll)], d_pbbt) +
                                   dot(Jacobi[g, (2lg+2lb+2ll+1):(2lg+2lb+3ll)], d_qbbf) +
                                   dot(Jacobi[g, (2lg+2lb+3ll+1):(2lg+2lb+4ll)], d_qbbt) <= 0) 
        end

        #real power max limit
        for g in (2lb+4ll+1+2therm_lim_ctr+lg):(2lb+4ll+2therm_lim_ctr+2lg)
            @constraint(m, gg[g] + dot(Jacobi[g, 1:lg],d_pg) +
                                   dot(Jacobi[g, lg+1:2lg],d_qg) +
                                   dot(Jacobi[g, 2lg+1:(2lg+lb)], d_vb) +
                                   dot(Jacobi[g, (2lg+lb+1):(2lg+2lb)],d_δb) +
                                   dot(Jacobi[g, (2lg+2lb+1):(2lg+2lb+ll)], d_pbbf) + 
                                   dot(Jacobi[g, (2lg+2lb+ll+1):(2lg+2lb+2ll)], d_pbbt) +
                                   dot(Jacobi[g, (2lg+2lb+2ll+1):(2lg+2lb+3ll)], d_qbbf) +
                                   dot(Jacobi[g, (2lg+2lb+3ll+1):(2lg+2lb+4ll)], d_qbbt) <= 0)
        end
        

        #reactive power min limit
        for g in (2lb+4ll+1+2therm_lim_ctr+2lg):(2lb+4ll+2therm_lim_ctr+3lg)
            @constraint(m, gg[g] + dot(Jacobi[g, 1:lg],d_pg) +
                                   dot(Jacobi[g, lg+1:2lg],d_qg) +
                                   dot(Jacobi[g, 2lg+1:(2lg+lb)], d_vb) +
                                   dot(Jacobi[g, (2lg+lb+1):(2lg+2lb)],d_δb) +
                                   dot(Jacobi[g, (2lg+2lb+1):(2lg+2lb+ll)], d_pbbf) + 
                                   dot(Jacobi[g, (2lg+2lb+ll+1):(2lg+2lb+2ll)], d_pbbt) +
                                   dot(Jacobi[g, (2lg+2lb+2ll+1):(2lg+2lb+3ll)], d_qbbf) +
                                   dot(Jacobi[g, (2lg+2lb+3ll+1):(2lg+2lb+4ll)], d_qbbt) <= 0)
        end
        
        #reactive power max limit
        for g in (2lb+4ll+1+2therm_lim_ctr+3lg):(2lb+4ll+2therm_lim_ctr+4lg)
            @constraint(m, gg[g] + dot(Jacobi[g, 1:lg],d_pg) +
                                   dot(Jacobi[g, lg+1:2lg],d_qg) +
                                   dot(Jacobi[g, 2lg+1:(2lg+lb)], d_vb) +
                                   dot(Jacobi[g, (2lg+lb+1):(2lg+2lb)],d_δb) +
                                   dot(Jacobi[g, (2lg+2lb+1):(2lg+2lb+ll)], d_pbbf) + 
                                   dot(Jacobi[g, (2lg+2lb+ll+1):(2lg+2lb+2ll)], d_pbbt) +
                                   dot(Jacobi[g, (2lg+2lb+2ll+1):(2lg+2lb+3ll)], d_qbbf) +
                                   dot(Jacobi[g, (2lg+2lb+3ll+1):(2lg+2lb+4ll)], d_qbbt) <= 0)
        end
        
        #voltage level min limit
        for b in (2lb+4ll+1+2therm_lim_ctr+4lg):(3lb+4ll+2therm_lim_ctr+4lg)
            @constraint(m, gg[b] + dot(Jacobi[b, 1:lg],d_pg) +
                                   dot(Jacobi[b, lg+1:2lg],d_qg) +
                                   dot(Jacobi[b, 2lg+1:(2lg+lb)], d_vb) +
                                   dot(Jacobi[b, (2lg+lb+1):(2lg+2lb)],d_δb) +
                                   dot(Jacobi[b, (2lg+2lb+1):(2lg+2lb+ll)], d_pbbf) + 
                                   dot(Jacobi[b, (2lg+2lb+ll+1):(2lg+2lb+2ll)], d_pbbt) +
                                   dot(Jacobi[b, (2lg+2lb+2ll+1):(2lg+2lb+3ll)], d_qbbf) +
                                   dot(Jacobi[b, (2lg+2lb+3ll+1):(2lg+2lb+4ll)], d_qbbt) <= 0)
        end
        
        #voltage level max limit
        for b in (3lb+4ll+1+2therm_lim_ctr+4lg):(4lb+4ll+2therm_lim_ctr+4lg)
            @constraint(m, gg[b] + dot(Jacobi[b, 1:lg],d_pg) +
                                   dot(Jacobi[b, lg+1:2lg],d_qg) +
                                   dot(Jacobi[b, 2lg+1:(2lg+lb)], d_vb) +
                                   dot(Jacobi[b, (2lg+lb+1):(2lg+2lb)],d_δb) +
                                   dot(Jacobi[b, (2lg+2lb+1):(2lg+2lb+ll)], d_pbbf) + 
                                   dot(Jacobi[b, (2lg+2lb+ll+1):(2lg+2lb+2ll)], d_pbbt) +
                                   dot(Jacobi[b, (2lg+2lb+2ll+1):(2lg+2lb+3ll)], d_qbbf) +
                                   dot(Jacobi[b, (2lg+2lb+3ll+1):(2lg+2lb+4ll)], d_qbbt) <= 0)
        end

        #slack bus voltage phase 
        for r = (4lb+4ll+1+2therm_lim_ctr+4lg)
            @constraint(m, gg[r] + dot(Jacobi[r, 1:lg],d_pg) +
                                   dot(Jacobi[r, lg+1:2lg],d_qg) +
                                   dot(Jacobi[r, 2lg+1:(2lg+lb)], d_vb) +
                                   dot(Jacobi[r, (2lg+lb+1):(2lg+2lb)],d_δb) +
                                   dot(Jacobi[r, (2lg+2lb+1):(2lg+2lb+ll)], d_pbbf) + 
                                   dot(Jacobi[r, (2lg+2lb+ll+1):(2lg+2lb+2ll)], d_pbbt) +
                                   dot(Jacobi[r, (2lg+2lb+2ll+1):(2lg+2lb+3ll)], d_qbbf) +
                                   dot(Jacobi[r, (2lg+2lb+3ll+1):(2lg+2lb+4ll)], d_qbbt) == 0)
        end
        
        @assert size(Jacobi, 1) == (4lb+4ll+1+2therm_lim_ctr+4lg)
     
        # trust region constraints        
        @constraint(m, d_pg .<= ρ)
        @constraint(m, d_qg .<= ρ)
        @constraint(m, d_vb .<= ρ)
        @constraint(m, d_δb .<= ρ)
        @constraint(m, d_pbbf .<= ρ)
        @constraint(m, d_pbbt .<= ρ)
        @constraint(m, d_qbbf .<= ρ)
        @constraint(m, d_qbbt .<= ρ)
        @constraint(m, d_pg .>= -ρ)
        @constraint(m, d_qg .>= -ρ)
        @constraint(m, d_vb .>= -ρ)
        @constraint(m, d_δb .>= -ρ)
        @constraint(m, d_pbbf .>= -ρ)
        @constraint(m, d_pbbt .>= -ρ)
        @constraint(m, d_qbbf .>= -ρ)
        @constraint(m, d_pbbt .>= -ρ)
  
   
        println("Iteration $i...")
        optimize!(m)
        @show termination_status(m)
        @show objective_value(m)
        
        
        diff_vars = vcat(value.(d_pg), value.(d_qg), value.(d_vb), value.(d_δb), [value.(d_pbbf[b1,b2]) for b1 in B for b2 in E_b_f[b1]], [value.(d_pbbt[b1,b2]) for b1 in B for b2 in E_b_f[b1]], [value.(d_qbbf[b1,b2]) for b1 in B for b2 in E_b_f[b1]], [value.(d_qbbt[b1,b2]) for b1 in B for b2 in E_b_f[b1]])
        
        # updating difference values
        prop_d_pg = pk .+ value.(d_pg)
        prop_d_qg = qk .+ value.(d_qg)
        prop_d_vb = vk .+ value.(d_vb)
        prop_d_δb = δk .+ value.(d_δb)      
        
        prop_d_pbbf = []
        prop_d_qbbf = []
        prop_d_pbbt = []
        prop_d_qbbt = []
        ctr = 0
        for b1 in B
            for b2 in E_b_f[b1]
                ctr +=1
                prop_d_pbbf = push!(prop_d_pbbf, pbbfk[ctr] + value.(d_pbbf[b1, b2]))
                prop_d_qbbf = push!(prop_d_qbbf, qbbfk[ctr] + value.(d_qbbf[b1, b2]))
                prop_d_pbbt = push!(prop_d_pbbt, pbbtk[ctr] + value.(d_pbbt[b1, b2]))
                prop_d_qbbt = push!(prop_d_qbbt, qbbtk[ctr] + value.(d_qbbt[b1, b2]))
            end
        end
    
        
        # convert all new points into vectors of float64 types
        prop_d_pg = convert(Vector{Float64},prop_d_pg)
        prop_d_qg = convert(Vector{Float64},prop_d_qg)
        prop_d_vb = convert(Vector{Float64},prop_d_vb)
        prop_d_δb = convert(Vector{Float64},prop_d_δb)
        prop_d_pbbf = convert(Vector{Float64},prop_d_pbbf)
        prop_d_pbbt = convert(Vector{Float64},prop_d_pbbt)
        prop_d_qbbf = convert(Vector{Float64},prop_d_qbbf)
        prop_d_qbbt = convert(Vector{Float64},prop_d_qbbt)

        prop_vals = [prop_d_pg;prop_d_qg;prop_d_vb;prop_d_δb;prop_d_pbbf;prop_d_pbbt;prop_d_qbbf;prop_d_qbbt]

 
        
        # merit function
        function mer(x::Vector) 
            newobjev = MOI.eval_objective(d, x)
            ce = zeros(length(all_nonlinear_constraints(nlmod)))
            MOI.eval_constraint(d,ce,x)
            
            eq_rows = ce[1:(2lb+4ll)]
            eq_rows = vcat(eq_rows, ce[end])
            ineq_rows = ce[(2lb+4ll+1):end-1]
            @assert length([eq_rows;ineq_rows])==length(ce)
            max_ineq_rows = []
            for i in 1:length(ineq_rows)
                maxval = max(ineq_rows[i], 0)
                max_ineq_rows = push!(max_ineq_rows, maxval)
            end
            
            merit = newobjev + γ * (norm(eq_rows) + norm(max_ineq_rows))
            return merit
        end
        
        println("mer(xk) = $(mer(vals))")
        println("mer(prop_xk) = $(mer(prop_vals))")
        
        if termination_status(m) == MathOptInterface.LOCALLY_INFEASIBLE
        elseif mer(prop_vals) < mer(vals)
            println("merit condition: success")
            pk =prop_d_pg
            qk =prop_d_qg
            vk = prop_d_vb
            δk =prop_d_δb
            pbbfk = prop_d_pbbf
            pbbtk = prop_d_pbbt
            qbbfk = prop_d_qbbf
            qbbtk = prop_d_qbbt
            ρ += 1*ρ
            
        else
            println("merit condition: failed")
#             ρ -= 0.5*ρ
            max_step = norm(diff_vars, Inf) # more aggressive trust region
            ρ = max_step/2
        end
    
        #restoration phase:
        if termination_status(m) == MathOptInterface.LOCALLY_INFEASIBLE || termination_status(m) == MathOptInterface.NUMERICAL_ERROR
            pk, qk, vk, δk, pbbfk, pbbtk, qbbfk, qbbtk, ρ = restoration_phase(pk, qk, vk, δk, pbbfk, pbbtk, qbbfk, qbbtk, ρ)
        end
        
        
        #convergence test
        tol = 1e-3
        if all(y->(y<=tol), diff_vars)
            @show ρ
            break
        end
    end
    
end


SLP_MF_OPF (generic function with 1 method)

In [10]:
pk = [1,1,1]
qk=[1,1,1]
vk=[1,1,1,1,1,1,1,1,1]
δk=[0,0,0,0,0,0,0,0,0]
pbbfk=[1,1,1,1,1,1,1,1,1]
pbbtk=[1,1,1,1,1,1,1,1,1]
qbbfk=[1,1,1,1,1,1,1,1,1]
qbbtk=[1,1,1,1,1,1,1,1,1]

SLP_MF_OPF(pk, qk, vk, δk, pbbfk, pbbtk, qbbfk, qbbtk, 4, 10; maxiter=20)

Iteration 1...
This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:      199
Number of nonzeros in inequality constraint Jacobian.:      186
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:       60
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:       55
Total number of inequality constraints...............:      168
        inequality constraints with only lower bounds:       60
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:      108

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  4.9800000e+03 3.25e+00 9.37e-01  -1.0 0.00e+00    -  0.00

Iteration 5...
This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:      223
Number of nonzeros in inequality constraint Jacobian.:      186
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:       60
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:       55
Total number of inequality constraints...............:      168
        inequality constraints with only lower bounds:       60
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:      108

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  4.8052346e+03 2.41e+00 1.08e+00  -1.0 0.00e+00    -  0.00

restoration merit condition: failed
restoration merit condition: failed
restoration merit condition: failed
restoration merit condition: failed
restoration merit condition: failed
restoration merit condition: failed
restoration merit condition: failed
restoration merit condition: failed
restoration merit condition: failed
restoration merit condition: failed
restoration merit condition: failed


LoadError: InterruptException:

In [10]:
pk = [10,10,10]
qk=[5,5,5]
vk = [1,1,1,1,1,1,1,1,1]
δk=[0,0,0,0,0,0,0,0,0]
pbbfk=[1,1,1,1,1,1,1,1,1]
pbbtk=[1,1,1,1,1,1,1,1,1]
qbbfk=[1,1,1,1,1,1,1,1,1]
qbbtk=[1,1,1,1,1,1,1,1,1]
SLP_MF_OPF(pk, qk, vk, δk, pbbfk, pbbtk, qbbfk, qbbtk, 4, 1; maxiter=80)

Iteration 1...
This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:      199
Number of nonzeros in inequality constraint Jacobian.:      186
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:       60
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:       55
Total number of inequality constraints...............:      168
        inequality constraints with only lower bounds:       60
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:      108

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  3.2578500e+05 9.00e+00 8.34e-01  -1.0 0.00e+00    -  0.00

This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:      223
Number of nonzeros in inequality constraint Jacobian.:      186
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:       60
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:       55
Total number of inequality constraints...............:      168
        inequality constraints with only lower bounds:       60
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:      108

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  9.1893676e+03 1.87e+01 4.57e+00  -1.0 0.00e+00    -  0.00e+00 0.00e+00  

mer(prop_xk) = 9445.517203933912
Solving restoration problem...
restoration merit condition: failed
restoration merit condition: failed
restoration merit condition: failed
restoration merit condition: failed
restoration merit condition: failed
restoration merit condition: failed
restoration merit condition: failed
restoration merit condition: failed
restoration merit condition: failed
restoration merit condition: failed
restoration merit condition: failed
restoration merit condition: failed
restoration merit condition: failed
restoration merit condition: failed
restoration merit condition: failed
restoration merit condition: failed
restoration merit condition: failed
restoration merit condition: failed
restoration merit condition: failed
restoration merit condition: failed
restoration merit condition: failed
restoration merit condition: failed
restoration merit condition: failed
restoration merit condition: failed
restoration merit condition: failed
restoration merit condition: failed


LoadError: InterruptException:

In [17]:
pk = [
 0.9009956060478431
 1.344415284478629
 0.9431268288272774]
qk = [
 0.6920306950513981
 0.4988499647310056
 0.3964768224657025]
vk =[ 
 1.1000000098672171
 1.100000009806191
 1.1000000096555398
 1.0648084897201466
 1.0405550528720546
 1.0800478711874757
 1.0574698991123597
 1.0743752385168213
 1.0276264475028547]
δk = [
  2.9709259043065792e-30
  0.08744073734526106
  0.05632974798987839
 -0.04432238940121758
 -0.07022513991579339
  0.009793776630580594
 -0.02180023771754267
  0.016281487783394614
 -0.08318352239305883]
pbbfk = [0.900996
  0.943127
0.352503
 -0.550084
  0.381823
 -0.620082
 -1.34442
 0.721207
  -0.54435]
pbbtk= [-0.900996
 -0.943127
 -0.349916
  0.561303
 -0.379918
 0.623208
 1.34442
 -0.70565
  0.548493]
qbbfk = [0.692031
  0.396477
 0.219613
  -0.0943829
 0.202496
  -0.163646
  -0.392636
  0.202509
  -0.37576]
qbbtk =  [-0.630589
  -0.345786
   -0.205617
 0.143291
 -0.186354
 0.190127
 0.49885
 -0.12424
 0.410976]
SLP_MF_OPF(pk, qk, vk, δk, pbbfk, pbbtk, qbbfk, qbbtk, 0.01, 1; maxiter=20)

Iteration 1...
This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:      223
Number of nonzeros in inequality constraint Jacobian.:      186
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:       60
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:       55
Total number of inequality constraints...............:      168
        inequality constraints with only lower bounds:       60
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:      108

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  5.3100704e+03 5.00e-06 2.10e-01  -1.0 0.00e+00    -  0.00

# SLP with a filter function

In [11]:
function SLP_F_OPF(pk, qk, vk, δk, pbbfk, pbbtk, qbbfk, qbbtk, ρ; maxiter=100)

    filt = []
    is_converged = false
    
    # nonlinear model for moi evaluation
    nlmod = Model()
    set_string_names_on_creation(nlmod, false)
#     set_silent(nlmod)
    
    @variable(nlmod, pg[g in G]) # real power generation
    @variable(nlmod, qg[g in G]) # reactive power generation
    @variable(nlmod, vb[b in B],(start=1.0)) # bus voltage
    @variable(nlmod, δb[b in B],(start=0.0)) # bus voltage phase
    @variable(nlmod, pbbf[b1 in B, b2 in E_b_f[b1]]) # line real power (from)
    @variable(nlmod, pbbt[b1 in B, b2 in E_b_f[b1]]) # line real power (to)
    @variable(nlmod, qbbf[b1 in B, b2 in E_b_f[b1]]) # line reactive power (from)
    @variable(nlmod, qbbt[b1 in B, b2 in E_b_f[b1]]) # line reactive power (to)

    
    @NLobjective(nlmod, Min, sum(100^2*gen_c[g][1] * pg[g]^2 + 100*gen_c[g][2] * pg[g] + gen_c[g][3] for g in G))
    
    # kcl for real power
    for b in B
        @NLconstraint(nlmod, sum(pg[g] for g in β_g[b]) == demand_real[b] +
                                                           sum(pbbf[b,b_] for b_ in E_b_f[b]) +
                                                           sum(pbbt[b_,b] for b_ in E_b_t[b]) +
                                                           shunt_conductance[b] * (vb[b])^2)
    end
    
    # kcl for reactive power
    for b in B
        @NLconstraint(nlmod, sum(qg[g] for g in β_g[b]) == demand_reactive[b] + sum(qbbf[b,b_] for b_ in E_b_f[b]) +
                                sum(qbbt[b_,b] for b_ in E_b_t[b]) - shunt_susceptance[b] * (vb[b])^2)
    end
    

    # kvl real power from
    for b1 in B
        for b2 in E_b_f[b1]
            @NLconstraint(nlmod, line_con[(b1,b2)]* (vb[b1])^2 - vb[b1]*vb[b2]*
                    (line_con[(b1,b2)] * cos(δb[b1] - δb[b2]) + line_sus[(b1,b2)] * sin(δb[b1] - δb[b2])) == pbbf[b1, b2])
        end
    end
    
    # kvl real power to
    for b1 in B
        for b2 in E_b_f[b1]
            @NLconstraint(nlmod, line_con[(b2,b1)]* (vb[b2])^2 - vb[b2]*vb[b1]*
                    (line_con[(b2,b1)] * cos(δb[b2] - δb[b1]) + line_sus[(b2,b1)] * sin(δb[b2] - δb[b1])) == pbbt[b1, b2])
        end
    end
    
    # kvl reactive power from
    for b1 in B
        for b2 in E_b_f[b1]
            @NLconstraint(nlmod, -line_sus[(b1,b2)]* (vb[b1])^2 - vb[b1]*vb[b2]*
                        (line_con[(b1,b2)] * sin(δb[b1] - δb[b2]) - line_sus[(b1,b2)] * cos(δb[b1] - δb[b2])) == qbbf[b1, b2])
        end
    end
        
    # kvl reactive power to
    for b1 in B
        for b2 in E_b_f[b1]
            @NLconstraint(nlmod, -line_sus[(b2,b1)]* (vb[b2])^2 - vb[b2]*vb[b1]*
                        (line_con[(b2,b1)] * sin(δb[b2] - δb[b1]) - line_sus[(b2,b1)] * cos(δb[b2] - δb[b1])) == qbbt[b1, b2])
        end
    end
    
    # thermal limits from
    therm_lim_ctr = 0
    for b1 in B
        for b2 in E_b_f[b1]
            if line_therm_lim[(b1, b2)] != 0.0
                @NLconstraint(nlmod, (pbbf[b1, b2])^2 + (qbbf[b1, b2])^2 <= (0.01*line_therm_lim[(b1, b2)])^2)
                therm_lim_ctr += 1
            end
        end   
    end
    
    # thermal limits to
    for b1 in B
        for b2 in E_b_f[b1]
            if line_therm_lim[(b1, b2)] != 0.0
                @NLconstraint(nlmod, (pbbt[b1, b2])^2 + (qbbt[b1, b2])^2 <= (0.01*line_therm_lim[(b1, b2)])^2)
            end
        end
    end
    
    # real power min limit
    for g in G
        @NLconstraint(nlmod, pg_minus[g] - pg[g] <= 0) 
    end

    #real power max limit
    for g in G
        @NLconstraint(nlmod, pg[g] <= pg_plus[g])
    end

    #reactive power min limit
    for g in G
        @NLconstraint(nlmod, qg_minus[g] - qg[g] <= 0)
    end

    #reactive power max limit
    for g in G
        @NLconstraint(nlmod, qg[g] <= qg_plus[g])
    end
    
    #voltage level min limit
    for b in B
        @NLconstraint(nlmod, vb_minus[b] - vb[b] <= 0)
    end

    #voltage level max limit
    for b in B
        @NLconstraint(nlmod, vb[b] <= vb_plus[b])
    end

    #slack bus voltage phase 
    for i in 1:length(bus_types)
        if bus_types[i] == 3
            @NLconstraint(nlmod, δb[i] == 0)
        end
    end
    

    d = NLPEvaluator(nlmod)
    MOI.initialize(d, [:Jac,:Grad,:Hess])
    
    for i in 1:maxiter 
        if is_converged
            break
        end
        
        d = NLPEvaluator(nlmod)
        MOI.initialize(d, [:Jac,:Grad,:Hess])
        vals = [pk; qk; vk; δk; pbbfk; pbbtk; qbbfk; qbbtk]
        
        #Objective eval
        objEv=MOI.eval_objective(d, pk)

        #Objective gradient
        df=zeros(length(pk))
        MOI.eval_objective_gradient(d,df,pk)

        #Constraint eval
        gg = zeros(length(all_nonlinear_constraints(nlmod)))
        MOI.eval_constraint(d,gg,vals)

        #Jacobian eval
        JStr=MOI.jacobian_structure(d)
        J=zeros(length(JStr))
        MOI.eval_constraint_jacobian(d,J,vals)

        jr = Vector{Int64}(undef, length(J)); jc = Vector{Int64}(undef, length(J))
        for i in 1:length(JStr)
            jr[i] = JStr[i][1] |> Int
            jc[i] = JStr[i][2] |> Int
        end
        Jacobeval = sparse(jr, jc, J) 
        Jacobi = Matrix(Jacobeval)

            
        # linear model    
        m = Model(Ipopt.Optimizer)
#         m = Model(Gurobi.Optimizer)
        set_string_names_on_creation(m, false)
        set_silent(m)
        @variable(m, d_pg[g in G])
        @variable(m, d_qg[g in G])
        @variable(m, d_vb[b in B])
        @variable(m, d_δb[b in B])
        @variable(m, d_pbbf[b1 in B, b2 in E_b_f[b1]])
        @variable(m, d_pbbt[b1 in B, b2 in E_b_f[b1]])
        @variable(m, d_qbbf[b1 in B, b2 in E_b_f[b1]])
        @variable(m, d_qbbt[b1 in B, b2 in E_b_f[b1]])
        
        @objective(m, Min, objEv + df'*d_pg) 

        
        #kcl for real power:
        for b in B
            @constraint(m, gg[b] + dot(Jacobi[b, 1:lg],d_pg) +
                                   dot(Jacobi[b, lg+1:2lg],d_qg) +
                                   dot(Jacobi[b, 2lg+1:(2lg+lb)], d_vb) +
                                   dot(Jacobi[b, (2lg+lb+1):(2lg+2lb)],d_δb) +
                                   dot(Jacobi[b, (2lg+2lb+1):(2lg+2lb+ll)], d_pbbf) + 
                                   dot(Jacobi[b, (2lg+2lb+ll+1):(2lg+2lb+2ll)], d_pbbt) +
                                   dot(Jacobi[b, (2lg+2lb+2ll+1):(2lg+2lb+3ll)], d_qbbf) +
                                   dot(Jacobi[b, (2lg+2lb+3ll+1):(2lg+2lb+4ll)], d_qbbt) == 0)
        end
        

        #kcl for reactive power
        for b in (lb+1):2lb
            @constraint(m, gg[b] + dot(Jacobi[b, 1:lg],d_pg) +
                                   dot(Jacobi[b, lg+1:2lg],d_qg) +
                                   dot(Jacobi[b, 2lg+1:(2lg+lb)], d_vb) +
                                   dot(Jacobi[b, (2lg+lb+1):(2lg+2lb)],d_δb) +
                                   dot(Jacobi[b, (2lg+2lb+1):(2lg+2lb+ll)], d_pbbf) + 
                                   dot(Jacobi[b, (2lg+2lb+ll+1):(2lg+2lb+2ll)], d_pbbt) +
                                   dot(Jacobi[b, (2lg+2lb+2ll+1):(2lg+2lb+3ll)], d_qbbf) +
                                   dot(Jacobi[b, (2lg+2lb+3ll+1):(2lg+2lb+4ll)], d_qbbt) == 0)
        end


        #kvl real power from
        for l in (2lb+1):(2lb+ll)
            @constraint(m, gg[l] + dot(Jacobi[l, 1:lg],d_pg) +
                                   dot(Jacobi[l, lg+1:2lg],d_qg) +
                                   dot(Jacobi[l, 2lg+1:(2lg+lb)], d_vb) +
                                   dot(Jacobi[l, (2lg+lb+1):(2lg+2lb)],d_δb) +
                                   dot(Jacobi[l, (2lg+2lb+1):(2lg+2lb+ll)], d_pbbf) + 
                                   dot(Jacobi[l, (2lg+2lb+ll+1):(2lg+2lb+2ll)], d_pbbt) +
                                   dot(Jacobi[l, (2lg+2lb+2ll+1):(2lg+2lb+3ll)], d_qbbf) +
                                   dot(Jacobi[l, (2lg+2lb+3ll+1):(2lg+2lb+4ll)], d_qbbt) == 0)
        end
                
        #kvl real power to
        for l in (2lb+ll+1):(2lb+2ll)
            @constraint(m, gg[l] + dot(Jacobi[l, 1:lg],d_pg) +
                                   dot(Jacobi[l, lg+1:2lg],d_qg) +
                                   dot(Jacobi[l, 2lg+1:(2lg+lb)], d_vb) +
                                   dot(Jacobi[l, (2lg+lb+1):(2lg+2lb)],d_δb) +
                                   dot(Jacobi[l, (2lg+2lb+1):(2lg+2lb+ll)], d_pbbf) + 
                                   dot(Jacobi[l, (2lg+2lb+ll+1):(2lg+2lb+2ll)], d_pbbt) +
                                   dot(Jacobi[l, (2lg+2lb+2ll+1):(2lg+2lb+3ll)], d_qbbf) +
                                   dot(Jacobi[l, (2lg+2lb+3ll+1):(2lg+2lb+4ll)], d_qbbt) == 0)
        end
        
        #kvl reactive power from
        for l in (2lb+2ll+1):(2lb+3ll)
            @constraint(m, gg[l] + dot(Jacobi[l, 1:lg],d_pg) +
                                   dot(Jacobi[l, lg+1:2lg],d_qg) +
                                   dot(Jacobi[l, 2lg+1:(2lg+lb)], d_vb) +
                                   dot(Jacobi[l, (2lg+lb+1):(2lg+2lb)],d_δb) +
                                   dot(Jacobi[l, (2lg+2lb+1):(2lg+2lb+ll)], d_pbbf) + 
                                   dot(Jacobi[l, (2lg+2lb+ll+1):(2lg+2lb+2ll)], d_pbbt) +
                                   dot(Jacobi[l, (2lg+2lb+2ll+1):(2lg+2lb+3ll)], d_qbbf) +
                                   dot(Jacobi[l, (2lg+2lb+3ll+1):(2lg+2lb+4ll)], d_qbbt) == 0)
        end
        
        #kvl reactive power to
        for l in (2lb+3ll+1):(2lb+4ll)
            @constraint(m, gg[l] + dot(Jacobi[l, 1:lg],d_pg) +
                                   dot(Jacobi[l, lg+1:2lg],d_qg) +
                                   dot(Jacobi[l, 2lg+1:(2lg+lb)], d_vb) +
                                   dot(Jacobi[l, (2lg+lb+1):(2lg+2lb)],d_δb) +
                                   dot(Jacobi[l, (2lg+2lb+1):(2lg+2lb+ll)], d_pbbf) + 
                                   dot(Jacobi[l, (2lg+2lb+ll+1):(2lg+2lb+2ll)], d_pbbt) +
                                   dot(Jacobi[l, (2lg+2lb+2ll+1):(2lg+2lb+3ll)], d_qbbf) +
                                   dot(Jacobi[l, (2lg+2lb+3ll+1):(2lg+2lb+4ll)], d_qbbt) == 0)
        end       
       
        #thermal limits from
        for l in (2lb+4ll+1):(2lb+4ll+therm_lim_ctr)
            @constraint(m, gg[l] + dot(Jacobi[l, 1:lg],d_pg) +
                                   dot(Jacobi[l, lg+1:2lg],d_qg) +
                                   dot(Jacobi[l, 2lg+1:(2lg+lb)], d_vb) +
                                   dot(Jacobi[l, (2lg+lb+1):(2lg+2lb)],d_δb) +
                                   dot(Jacobi[l, (2lg+2lb+1):(2lg+2lb+ll)], d_pbbf) + 
                                   dot(Jacobi[l, (2lg+2lb+ll+1):(2lg+2lb+2ll)], d_pbbt) +
                                   dot(Jacobi[l, (2lg+2lb+2ll+1):(2lg+2lb+3ll)], d_qbbf) +
                                   dot(Jacobi[l, (2lg+2lb+3ll+1):(2lg+2lb+4ll)], d_qbbt) <= 0)
        end
        
        #thermal limits to
        for l in (2lb+4ll+1+therm_lim_ctr):(2lb+4ll+2therm_lim_ctr)
            @constraint(m, gg[l] + dot(Jacobi[l, 1:lg],d_pg) +
                                   dot(Jacobi[l, lg+1:2lg],d_qg) +
                                   dot(Jacobi[l, 2lg+1:(2lg+lb)], d_vb) +
                                   dot(Jacobi[l, (2lg+lb+1):(2lg+2lb)],d_δb) +
                                   dot(Jacobi[l, (2lg+2lb+1):(2lg+2lb+ll)], d_pbbf) + 
                                   dot(Jacobi[l, (2lg+2lb+ll+1):(2lg+2lb+2ll)], d_pbbt) +
                                   dot(Jacobi[l, (2lg+2lb+2ll+1):(2lg+2lb+3ll)], d_qbbf) +
                                   dot(Jacobi[l, (2lg+2lb+3ll+1):(2lg+2lb+4ll)], d_qbbt) <= 0)
        end
        
        # real power min limit
        for g in (2lb+4ll+1+2therm_lim_ctr):(2lb+4ll+2therm_lim_ctr+lg)
            @constraint(m, gg[g] + dot(Jacobi[g, 1:lg],d_pg) +
                                   dot(Jacobi[g, lg+1:2lg],d_qg) +
                                   dot(Jacobi[g, 2lg+1:(2lg+lb)], d_vb) +
                                   dot(Jacobi[g, (2lg+lb+1):(2lg+2lb)],d_δb) +
                                   dot(Jacobi[g, (2lg+2lb+1):(2lg+2lb+ll)], d_pbbf) + 
                                   dot(Jacobi[g, (2lg+2lb+ll+1):(2lg+2lb+2ll)], d_pbbt) +
                                   dot(Jacobi[g, (2lg+2lb+2ll+1):(2lg+2lb+3ll)], d_qbbf) +
                                   dot(Jacobi[g, (2lg+2lb+3ll+1):(2lg+2lb+4ll)], d_qbbt) <= 0) 
        end

        #real power max limit
        for g in (2lb+4ll+1+2therm_lim_ctr+lg):(2lb+4ll+2therm_lim_ctr+2lg)
            @constraint(m, gg[g] + dot(Jacobi[g, 1:lg],d_pg) +
                                   dot(Jacobi[g, lg+1:2lg],d_qg) +
                                   dot(Jacobi[g, 2lg+1:(2lg+lb)], d_vb) +
                                   dot(Jacobi[g, (2lg+lb+1):(2lg+2lb)],d_δb) +
                                   dot(Jacobi[g, (2lg+2lb+1):(2lg+2lb+ll)], d_pbbf) + 
                                   dot(Jacobi[g, (2lg+2lb+ll+1):(2lg+2lb+2ll)], d_pbbt) +
                                   dot(Jacobi[g, (2lg+2lb+2ll+1):(2lg+2lb+3ll)], d_qbbf) +
                                   dot(Jacobi[g, (2lg+2lb+3ll+1):(2lg+2lb+4ll)], d_qbbt) <= 0)
        end

        #reactive power min limit
        for g in (2lb+4ll+1+2therm_lim_ctr+2lg):(2lb+4ll+2therm_lim_ctr+3lg)
            @constraint(m, gg[g] + dot(Jacobi[g, 1:lg],d_pg) +
                                   dot(Jacobi[g, lg+1:2lg],d_qg) +
                                   dot(Jacobi[g, 2lg+1:(2lg+lb)], d_vb) +
                                   dot(Jacobi[g, (2lg+lb+1):(2lg+2lb)],d_δb) +
                                   dot(Jacobi[g, (2lg+2lb+1):(2lg+2lb+ll)], d_pbbf) + 
                                   dot(Jacobi[g, (2lg+2lb+ll+1):(2lg+2lb+2ll)], d_pbbt) +
                                   dot(Jacobi[g, (2lg+2lb+2ll+1):(2lg+2lb+3ll)], d_qbbf) +
                                   dot(Jacobi[g, (2lg+2lb+3ll+1):(2lg+2lb+4ll)], d_qbbt) <= 0)
        end
        
        #reactive power max limit
        for g in (2lb+4ll+1+2therm_lim_ctr+3lg):(2lb+4ll+2therm_lim_ctr+4lg)
            @constraint(m, gg[g] + dot(Jacobi[g, 1:lg],d_pg) +
                                   dot(Jacobi[g, lg+1:2lg],d_qg) +
                                   dot(Jacobi[g, 2lg+1:(2lg+lb)], d_vb) +
                                   dot(Jacobi[g, (2lg+lb+1):(2lg+2lb)],d_δb) +
                                   dot(Jacobi[g, (2lg+2lb+1):(2lg+2lb+ll)], d_pbbf) + 
                                   dot(Jacobi[g, (2lg+2lb+ll+1):(2lg+2lb+2ll)], d_pbbt) +
                                   dot(Jacobi[g, (2lg+2lb+2ll+1):(2lg+2lb+3ll)], d_qbbf) +
                                   dot(Jacobi[g, (2lg+2lb+3ll+1):(2lg+2lb+4ll)], d_qbbt) <= 0)
        end
        
        #voltage level min limit
        for b in (2lb+4ll+1+2therm_lim_ctr+4lg):(3lb+4ll+2therm_lim_ctr+4lg)
            @constraint(m, gg[b] + dot(Jacobi[b, 1:lg],d_pg) +
                                   dot(Jacobi[b, lg+1:2lg],d_qg) +
                                   dot(Jacobi[b, 2lg+1:(2lg+lb)], d_vb) +
                                   dot(Jacobi[b, (2lg+lb+1):(2lg+2lb)],d_δb) +
                                   dot(Jacobi[b, (2lg+2lb+1):(2lg+2lb+ll)], d_pbbf) + 
                                   dot(Jacobi[b, (2lg+2lb+ll+1):(2lg+2lb+2ll)], d_pbbt) +
                                   dot(Jacobi[b, (2lg+2lb+2ll+1):(2lg+2lb+3ll)], d_qbbf) +
                                   dot(Jacobi[b, (2lg+2lb+3ll+1):(2lg+2lb+4ll)], d_qbbt) <= 0)
        end
        
        #voltage level max limit
        for b in (3lb+4ll+1+2therm_lim_ctr+4lg):(4lb+4ll+2therm_lim_ctr+4lg)
            @constraint(m, gg[b] + dot(Jacobi[b, 1:lg],d_pg) +
                                   dot(Jacobi[b, lg+1:2lg],d_qg) +
                                   dot(Jacobi[b, 2lg+1:(2lg+lb)], d_vb) +
                                   dot(Jacobi[b, (2lg+lb+1):(2lg+2lb)],d_δb) +
                                   dot(Jacobi[b, (2lg+2lb+1):(2lg+2lb+ll)], d_pbbf) + 
                                   dot(Jacobi[b, (2lg+2lb+ll+1):(2lg+2lb+2ll)], d_pbbt) +
                                   dot(Jacobi[b, (2lg+2lb+2ll+1):(2lg+2lb+3ll)], d_qbbf) +
                                   dot(Jacobi[b, (2lg+2lb+3ll+1):(2lg+2lb+4ll)], d_qbbt) <= 0)
        end

        #slack bus voltage phase 
        for i in (4lb+4ll+1+2therm_lim_ctr+4lg)
            @constraint(m, gg[i] + dot(Jacobi[i, 1:lg],d_pg) +
                                   dot(Jacobi[i, lg+1:2lg],d_qg) +
                                   dot(Jacobi[i, 2lg+1:(2lg+lb)], d_vb) +
                                   dot(Jacobi[i, (2lg+lb+1):(2lg+2lb)],d_δb) +
                                   dot(Jacobi[i, (2lg+2lb+1):(2lg+2lb+ll)], d_pbbf) + 
                                   dot(Jacobi[i, (2lg+2lb+ll+1):(2lg+2lb+2ll)], d_pbbt) +
                                   dot(Jacobi[i, (2lg+2lb+2ll+1):(2lg+2lb+3ll)], d_qbbf) +
                                   dot(Jacobi[i, (2lg+2lb+3ll+1):(2lg+2lb+4ll)], d_qbbt) == 0)
        end
        
        @assert size(Jacobi, 1) == (4lb+4ll+1+2therm_lim_ctr+4lg)
     
        # trust region constraints        
        @constraint(m, d_pg .<= ρ)
        @constraint(m, d_qg .<= ρ)
        @constraint(m, d_vb .<= ρ)
        @constraint(m, d_δb .<= ρ)
        @constraint(m, d_pbbf .<= ρ)
        @constraint(m, d_pbbt .<= ρ)
        @constraint(m, d_qbbf .<= ρ)
        @constraint(m, d_qbbt .<= ρ)
        @constraint(m, d_pg .>= -ρ)
        @constraint(m, d_qg .>= -ρ)
        @constraint(m, d_vb .>= -ρ)
        @constraint(m, d_δb .>= -ρ)
        @constraint(m, d_pbbf .>= -ρ)
        @constraint(m, d_pbbt .>= -ρ)
        @constraint(m, d_qbbf .>= -ρ)
        @constraint(m, d_pbbt .>= -ρ)
  
   
        println("Iteration $i...")
        optimize!(m)
        @show termination_status(m)
        @show objective_value(m)

        
        # updating difference values
        prop_d_pg = pk .+ value.(d_pg)
        prop_d_qg = qk .+ value.(d_qg)
        prop_d_vb = vk .+ value.(d_vb)
        prop_d_δb = δk .+ value.(d_δb)      
        
        prop_d_pbbf = []
        prop_d_qbbf = []
        prop_d_pbbt = []
        prop_d_qbbt = []
        ctr = 0
        for b1 in B
            for b2 in E_b_f[b1]
                ctr +=1
                prop_d_pbbf = push!(prop_d_pbbf, pbbfk[ctr] + value.(d_pbbf[b1, b2]))
                prop_d_qbbf = push!(prop_d_qbbf, qbbfk[ctr] + value.(d_qbbf[b1, b2]))
                prop_d_pbbt = push!(prop_d_pbbt, pbbtk[ctr] + value.(d_pbbt[b1, b2]))
                prop_d_qbbt = push!(prop_d_qbbt, qbbtk[ctr] + value.(d_qbbt[b1, b2]))
            end
        end
    
        
        # convert all new points into vectors of float64 types
        prop_d_pg = convert(Vector{Float64},prop_d_pg)
        prop_d_qg = convert(Vector{Float64},prop_d_qg)
        prop_d_vb = convert(Vector{Float64},prop_d_vb)
        prop_d_δb = convert(Vector{Float64},prop_d_δb)
        prop_d_pbbf = convert(Vector{Float64},prop_d_pbbf)
        prop_d_pbbt = convert(Vector{Float64},prop_d_pbbt)
        prop_d_qbbf = convert(Vector{Float64},prop_d_qbbf)
        prop_d_qbbt = convert(Vector{Float64},prop_d_qbbt)

        prop_vals = [prop_d_pg;prop_d_qg;prop_d_vb;prop_d_δb;prop_d_pbbf;prop_d_pbbt;prop_d_qbbf;prop_d_qbbt]
 
        #functions for making the filter
        function fk(x::Vector{<:Real}) 
            newobjev = MOI.eval_objective(d, x)
            return newobjev
        end
        
        function hk(x::Vector{<:Real}) 
            ce = zeros(length(all_nonlinear_constraints(nlmod)))
            MOI.eval_constraint(d,ce,x)
            
            eq_rows = ce[1:(2lb+4ll)]
            eq_rows = vcat(eq_rows, ce[end])
            ineq_rows = ce[(2lb+4ll+1):end-1]
            @assert length([eq_rows;ineq_rows])==length(ce)
            
            max_ineq_rows = []
            for i in 1:length(ineq_rows)
                maxval = max(ineq_rows[i], 0)
                max_ineq_rows = push!(max_ineq_rows, maxval)
            end
            
            h = norm(eq_rows) + norm(max_ineq_rows)
            return h
        end
        
        println("f(propxk) = $(fk(prop_vals))")
        println("f(xk) = $(fk(vals))")
        
        diff_vars = vcat(value.(d_pg), value.(d_qg), value.(d_vb), value.(d_δb), [value.(d_pbbf[b1,b2]) for b1 in B for b2 in E_b_f[b1]], [value.(d_pbbt[b1,b2]) for b1 in B for b2 in E_b_f[b1]], [value.(d_qbbf[b1,b2]) for b1 in B for b2 in E_b_f[b1]], [value.(d_qbbt[b1,b2]) for b1 in B for b2 in E_b_f[b1]])
        
        better_obj = all(v -> fk(prop_vals) < v[1], filt)
        better_constr = all(v -> hk(prop_vals) < v[2], filt)
        
        if termination_status(m) == MathOptInterface.LOCALLY_INFEASIBLE
        elseif better_obj || better_constr
            println("filter: accept")
            pk =prop_d_pg
            qk =prop_d_qg
            vk = prop_d_vb
            δk =prop_d_δb
            pbbfk = prop_d_pbbf
            pbbtk = prop_d_pbbt
            qbbfk = prop_d_qbbf
            qbbtk = prop_d_qbbt
            ρ += 1*ρ
            
            vals = [pk; qk; vk; δk; pbbfk; pbbtk; qbbfk; qbbtk]
            filt = push!(filt, (fk(vals), hk(vals)))
        else
            println("filter: reject")
#             ρ -= 0.5*ρ
            max_step = norm(diff_vars, Inf) # more aggressive trust region
            ρ = max_step/3
            filt = push!(filt, (fk(prop_vals), hk(prop_vals)))
        end
        
         #restoration phase:
        if termination_status(m) == MathOptInterface.LOCALLY_INFEASIBLE || termination_status(m) == MathOptInterface.NUMERICAL_ERROR
            pk, qk, vk, δk, pbbfk, pbbtk, qbbfk, qbbtk, ρ = restoration_phase(pk, qk, vk, δk, pbbfk, pbbtk, qbbfk, qbbtk, ρ)
        end   
        
        #convergence test
        tol = 1e-2
        if all(y->(y<=tol), diff_vars)
            @show ρ
            break
        end
        
    end
end

SLP_F_OPF (generic function with 1 method)

In [12]:
pk = [0.95, 0.95, 0.95]
qk = [0.5,0.5,0.5]
vk =[1,1,1,1,1,1,1,1,1]
δk = [0,0,0,0,0,0,0,0,0]
pbbfk = [0.9
  0.9
0.3
 -0.5
  0.4
 -0.6
 -1.3
 0.7
  -0.5]
pbbtk= [-0.9
 -0.9
 -0.3
  0.56
 -0.3
 0.6
 1.3
 -0.7
  0.5]
qbbfk = [0.7
  0.4
 0.2
  -0.0
 0.2
  -0.1
  -0.4
  0.2
  -0.4]
qbbtk =  [-0.6
  -0.3
   -0.2
 0.1
 -0.2
 0.2
 0.5
 -0.1
 0.4]
SLP_F_OPF(pk, qk, vk, δk, pbbfk, pbbtk, qbbfk, qbbtk, 1; maxiter=40)

Iteration 1...
termination_status(m) = MathOptInterface.LOCALLY_SOLVED
objective_value(m) = 4532.0624915795
f(propxk) = 6204.375026265014
f(xk) = 4634.4375
filter: accept
Iteration 2...
termination_status(m) = MathOptInterface.LOCALLY_SOLVED
objective_value(m) = 637.2809176270838
f(propxk) = 8013.13680440398
f(xk) = 6204.375026265014
filter: reject
Iteration 3...
termination_status(m) = MathOptInterface.LOCALLY_INFEASIBLE
objective_value(m) = 4783.615639910069
f(propxk) = 5593.409161391852
f(xk) = 6204.375026265014
Solving restoration problem...
restoration merit condition: failed
restoration merit condition: failed
Iteration 4...
termination_status(m) = MathOptInterface.LOCALLY_SOLVED
objective_value(m) = 4060.2096453416457
f(propxk) = 5334.519347291639
f(xk) = 6204.375026265014
filter: accept
Iteration 5...
termination_status(m) = MathOptInterface.LOCALLY_SOLVED
objective_value(m) = 4521.832358127938
f(propxk) = 6978.768434510825
f(xk) = 5334.519347291639
filter: reject
Iteration 6..

LoadError: InterruptException:

# SQP with a merit function

In [20]:
function SQP_MF_OPF(pk, qk, vk, δk, pbbfk, pbbtk, qbbfk, qbbtk, λk, ρ, γ; maxiter=100)

    is_converged = false
    
    # nonlinear model for moi evaluation
    nlmod = Model()
    set_string_names_on_creation(nlmod, false)
    set_silent(nlmod)
    
    @variable(nlmod, pg[g in G]) # real power generation
    @variable(nlmod, qg[g in G]) # reactive power generation
    @variable(nlmod, vb[b in B],(start=1.0)) # bus voltage
    @variable(nlmod, δb[b in B],(start=0.0)) # bus voltage phase
    @variable(nlmod, pbbf[b1 in B, b2 in E_b_f[b1]]) # line real power (from)
    @variable(nlmod, pbbt[b1 in B, b2 in E_b_f[b1]]) # line real power (to)
    @variable(nlmod, qbbf[b1 in B, b2 in E_b_f[b1]]) # line reactive power (from)
    @variable(nlmod, qbbt[b1 in B, b2 in E_b_f[b1]]) # line reactive power (to)

    
    @NLobjective(nlmod, Min, sum(100^2*gen_c[g][1] * pg[g]^2 + 100*gen_c[g][2] * pg[g] + gen_c[g][3] for g in G))
    
    # kcl for real power
    for b in B
        @NLconstraint(nlmod, sum(pg[g] for g in β_g[b]) == demand_real[b] +
                                                           sum(pbbf[b,b_] for b_ in E_b_f[b]) +
                                                           sum(pbbt[b_,b] for b_ in E_b_t[b]) +
                                                           shunt_conductance[b] * (vb[b])^2)
    end
    
    # kcl for reactive power
    for b in B
        @NLconstraint(nlmod, sum(qg[g] for g in β_g[b]) == demand_reactive[b] + sum(qbbf[b,b_] for b_ in E_b_f[b]) +
                                sum(qbbt[b_,b] for b_ in E_b_t[b]) - shunt_susceptance[b] * (vb[b])^2)
    end
    

    # kvl real power from
    for b1 in B
        for b2 in E_b_f[b1]
            @NLconstraint(nlmod, line_con[(b1,b2)]* (vb[b1])^2 - vb[b1]*vb[b2]*
                    (line_con[(b1,b2)] * cos(δb[b1] - δb[b2]) + line_sus[(b1,b2)] * sin(δb[b1] - δb[b2])) == pbbf[b1, b2])
        end
    end
    
    # kvl real power to
    for b1 in B
        for b2 in E_b_f[b1]
            @NLconstraint(nlmod, line_con[(b2,b1)]* (vb[b2])^2 - vb[b2]*vb[b1]*
                    (line_con[(b2,b1)] * cos(δb[b2] - δb[b1]) + line_sus[(b2,b1)] * sin(δb[b2] - δb[b1])) == pbbt[b1, b2])
        end
    end
    
    # kvl reactive power from
    for b1 in B
        for b2 in E_b_f[b1]
            @NLconstraint(nlmod, -line_sus[(b1,b2)]* (vb[b1])^2 - vb[b1]*vb[b2]*
                        (line_con[(b1,b2)] * sin(δb[b1] - δb[b2]) - line_sus[(b1,b2)] * cos(δb[b1] - δb[b2])) == qbbf[b1, b2])
        end
    end
        
    # kvl reactive power to
    for b1 in B
        for b2 in E_b_f[b1]
            @NLconstraint(nlmod, -line_sus[(b2,b1)]* (vb[b2])^2 - vb[b2]*vb[b1]*
                        (line_con[(b2,b1)] * sin(δb[b2] - δb[b1]) - line_sus[(b2,b1)] * cos(δb[b2] - δb[b1])) == qbbt[b1, b2])
        end
    end
    
    # thermal limits from
    therm_lim_ctr = 0
    for b1 in B
        for b2 in E_b_f[b1]
            if line_therm_lim[(b1, b2)] != 0.0
                @NLconstraint(nlmod, (pbbf[b1, b2])^2 + (qbbf[b1, b2])^2 <= (0.01*line_therm_lim[(b1, b2)])^2)
                therm_lim_ctr += 1
            end
        end   
    end
    
    # thermal limits to
    for b1 in B
        for b2 in E_b_f[b1]
            if line_therm_lim[(b1, b2)] != 0.0
                @NLconstraint(nlmod, (pbbt[b1, b2])^2 + (qbbt[b1, b2])^2 <= (0.01*line_therm_lim[(b1, b2)])^2)
            end
        end
    end
    
   # real power min limit
    for g in G
        @NLconstraint(nlmod, pg_minus[g] - pg[g] <= 0) 
    end

    #real power max limit
    for g in G
        @NLconstraint(nlmod, pg[g] <= pg_plus[g])
    end

    #reactive power min limit
    for g in G
        @NLconstraint(nlmod, qg_minus[g] - qg[g] <= 0)
    end

    #reactive power max limit
    for g in G
        @NLconstraint(nlmod, qg[g] <= qg_plus[g])
    end
    
    #voltage level min limit
    for b in B
        @NLconstraint(nlmod, vb_minus[b] - vb[b] <= 0)
    end

    #voltage level max limit
    for b in B
        @NLconstraint(nlmod, vb[b] <= vb_plus[b])
    end

    #slack bus voltage phase 
    for i in 1:length(bus_types)
        if bus_types[i] == 3
            @NLconstraint(nlmod, δb[i] == 0)
        end
    end
    

    d = NLPEvaluator(nlmod)
    MOI.initialize(d, [:Jac,:Grad,:Hess])
    
    for i in 1:maxiter 
        if is_converged
            break
        end
        
        d = NLPEvaluator(nlmod)
        MOI.initialize(d, [:Jac,:Grad,:Hess])
        
        vals = [pk; qk; vk; δk; pbbfk; pbbtk; qbbfk; qbbtk]
        #Objective eval
        objEv=MOI.eval_objective(d, pk) 

        #Objective gradient
        df=zeros(length(pk))
        MOI.eval_objective_gradient(d,df,pk)

        #Constraint eval
        gg = zeros(length(all_nonlinear_constraints(nlmod)))
        MOI.eval_constraint(d,gg,vals)
        
        #Jacobian eval
        JStr=MOI.jacobian_structure(d)
        J=zeros(length(JStr))
        MOI.eval_constraint_jacobian(d,J,vals)

        jr = Vector{Int64}(undef, length(J)); jc = Vector{Int64}(undef, length(J))
        for i in 1:length(JStr)
            jr[i] = JStr[i][1] |> Int
            jc[i] = JStr[i][2] |> Int
        end
        Jacobeval = sparse(jr, jc, J) 
        Jacobi = Matrix(Jacobeval)
        
        #Hessian-of-the-Lagrangian eval
        HStr=MOI.hessian_lagrangian_structure(d)
        H=zeros(length(HStr))
        MOI.eval_hessian_lagrangian(d,H,Float64.(vals),1.0,λk)
        
        
        hr = Vector{Int64}(undef, length(H)); hc = Vector{Int64}(undef, length(H))
        for i in 1:length(HStr)
            hr[i] = HStr[i][1] |> Int
            hc[i] = HStr[i][2] |> Int
        end
        
        HessLag = sparse(hr, hc, H)
        
        # linear model    
        m = Model(Ipopt.Optimizer)
        set_optimizer_attribute(m, "tol", 10^(-10))
#         m = Model(Gurobi.Optimizer)
#         set_silent(m)
        @variable(m, d_pg[g in G])
        @variable(m, d_qg[g in G])
        @variable(m, d_vb[b in B])
        @variable(m, d_δb[b in B])
        @variable(m, d_pbbf[b1 in B, b2 in E_b_f[b1]])
        @variable(m, d_pbbt[b1 in B, b2 in E_b_f[b1]])
        @variable(m, d_qbbf[b1 in B, b2 in E_b_f[b1]])
        @variable(m, d_qbbt[b1 in B, b2 in E_b_f[b1]])
        
        ds = vcat(d_pg, d_qg, d_vb, d_δb, [d_pbbf[b1,b2] for b1 in B for b2 in E_b_f[b1]], [d_pbbt[b1,b2] for b1 in B for b2 in E_b_f[b1]], [d_qbbf[b1,b2] for b1 in B for b2 in E_b_f[b1]], [d_qbbt[b1,b2] for b1 in B for b2 in E_b_f[b1]])
        
        @objective(m, Min, objEv + df'*d_pg + 0.5*ds'*(HessLag*ds)) 
        
        #kcl for real power:
        @constraint(m, kcl_p[b in B], gg[b] + dot(Jacobi[b, 1:lg],d_pg) +
                               dot(Jacobi[b, lg+1:2lg],d_qg) +
                               dot(Jacobi[b, 2lg+1:(2lg+lb)], d_vb) +
                               dot(Jacobi[b, (2lg+lb+1):(2lg+2lb)],d_δb) +
                               dot(Jacobi[b, (2lg+2lb+1):(2lg+2lb+ll)], d_pbbf) + 
                               dot(Jacobi[b, (2lg+2lb+ll+1):(2lg+2lb+2ll)], d_pbbt) +
                               dot(Jacobi[b, (2lg+2lb+2ll+1):(2lg+2lb+3ll)], d_qbbf) +
                               dot(Jacobi[b, (2lg+2lb+3ll+1):(2lg+2lb+4ll)], d_qbbt) == 0)
        

        #kcl for reactive power
        @constraint(m, kcl_q[b in (lb+1):2lb], gg[b] + dot(Jacobi[b, 1:lg],d_pg) +
                               dot(Jacobi[b, lg+1:2lg],d_qg) +
                               dot(Jacobi[b, 2lg+1:(2lg+lb)], d_vb) +
                               dot(Jacobi[b, (2lg+lb+1):(2lg+2lb)],d_δb) +
                               dot(Jacobi[b, (2lg+2lb+1):(2lg+2lb+ll)], d_pbbf) + 
                               dot(Jacobi[b, (2lg+2lb+ll+1):(2lg+2lb+2ll)], d_pbbt) +
                               dot(Jacobi[b, (2lg+2lb+2ll+1):(2lg+2lb+3ll)], d_qbbf) +
                               dot(Jacobi[b, (2lg+2lb+3ll+1):(2lg+2lb+4ll)], d_qbbt) == 0)


        #kvl real power from
        @constraint(m, kvl_pf[l in (2lb+1):(2lb+ll)], gg[l] + dot(Jacobi[l, 1:lg],d_pg) +
                               dot(Jacobi[l, lg+1:2lg],d_qg) +
                               dot(Jacobi[l, 2lg+1:(2lg+lb)], d_vb) +
                               dot(Jacobi[l, (2lg+lb+1):(2lg+2lb)],d_δb) +
                               dot(Jacobi[l, (2lg+2lb+1):(2lg+2lb+ll)], d_pbbf) + 
                               dot(Jacobi[l, (2lg+2lb+ll+1):(2lg+2lb+2ll)], d_pbbt) +
                               dot(Jacobi[l, (2lg+2lb+2ll+1):(2lg+2lb+3ll)], d_qbbf) +
                               dot(Jacobi[l, (2lg+2lb+3ll+1):(2lg+2lb+4ll)], d_qbbt) == 0)
        
                
        #kvl real power to
        @constraint(m, kvl_pt[l in (2lb+ll+1):(2lb+2ll)], gg[l] + dot(Jacobi[l, 1:lg],d_pg) +
                               dot(Jacobi[l, lg+1:2lg],d_qg) +
                               dot(Jacobi[l, 2lg+1:(2lg+lb)], d_vb) +
                               dot(Jacobi[l, (2lg+lb+1):(2lg+2lb)],d_δb) +
                               dot(Jacobi[l, (2lg+2lb+1):(2lg+2lb+ll)], d_pbbf) + 
                               dot(Jacobi[l, (2lg+2lb+ll+1):(2lg+2lb+2ll)], d_pbbt) +
                               dot(Jacobi[l, (2lg+2lb+2ll+1):(2lg+2lb+3ll)], d_qbbf) +
                               dot(Jacobi[l, (2lg+2lb+3ll+1):(2lg+2lb+4ll)], d_qbbt) == 0)
        
        #kvl reactive power from
        @constraint(m, kvl_qf[l in (2lb+2ll+1):(2lb+3ll)], gg[l] + dot(Jacobi[l, 1:lg],d_pg) +
                               dot(Jacobi[l, lg+1:2lg],d_qg) +
                               dot(Jacobi[l, 2lg+1:(2lg+lb)], d_vb) +
                               dot(Jacobi[l, (2lg+lb+1):(2lg+2lb)],d_δb) +
                               dot(Jacobi[l, (2lg+2lb+1):(2lg+2lb+ll)], d_pbbf) + 
                               dot(Jacobi[l, (2lg+2lb+ll+1):(2lg+2lb+2ll)], d_pbbt) +
                               dot(Jacobi[l, (2lg+2lb+2ll+1):(2lg+2lb+3ll)], d_qbbf) +
                               dot(Jacobi[l, (2lg+2lb+3ll+1):(2lg+2lb+4ll)], d_qbbt) == 0)
        
        #kvl reactive power to
        @constraint(m, kvl_qt[l in (2lb+3ll+1):(2lb+4ll)], gg[l] + dot(Jacobi[l, 1:lg],d_pg) +
                               dot(Jacobi[l, lg+1:2lg],d_qg) +
                               dot(Jacobi[l, 2lg+1:(2lg+lb)], d_vb) +
                               dot(Jacobi[l, (2lg+lb+1):(2lg+2lb)],d_δb) +
                               dot(Jacobi[l, (2lg+2lb+1):(2lg+2lb+ll)], d_pbbf) + 
                               dot(Jacobi[l, (2lg+2lb+ll+1):(2lg+2lb+2ll)], d_pbbt) +
                               dot(Jacobi[l, (2lg+2lb+2ll+1):(2lg+2lb+3ll)], d_qbbf) +
                               dot(Jacobi[l, (2lg+2lb+3ll+1):(2lg+2lb+4ll)], d_qbbt) == 0)     
       
        #thermal limits from
        @constraint(m, tl_f[l in (2lb+4ll+1):(2lb+4ll+therm_lim_ctr)], gg[l] + dot(Jacobi[l, 1:lg],d_pg) +
                               dot(Jacobi[l, lg+1:2lg],d_qg) +
                               dot(Jacobi[l, 2lg+1:(2lg+lb)], d_vb) +
                               dot(Jacobi[l, (2lg+lb+1):(2lg+2lb)],d_δb) +
                               dot(Jacobi[l, (2lg+2lb+1):(2lg+2lb+ll)], d_pbbf) + 
                               dot(Jacobi[l, (2lg+2lb+ll+1):(2lg+2lb+2ll)], d_pbbt) +
                               dot(Jacobi[l, (2lg+2lb+2ll+1):(2lg+2lb+3ll)], d_qbbf) +
                               dot(Jacobi[l, (2lg+2lb+3ll+1):(2lg+2lb+4ll)], d_qbbt) <= 0)
        
        #thermal limits to
        @constraint(m, tl_t[l in (2lb+4ll+1+therm_lim_ctr):(2lb+4ll+2therm_lim_ctr)], gg[l] + dot(Jacobi[l, 1:lg],d_pg) +
                               dot(Jacobi[l, lg+1:2lg],d_qg) +
                               dot(Jacobi[l, 2lg+1:(2lg+lb)], d_vb) +
                               dot(Jacobi[l, (2lg+lb+1):(2lg+2lb)],d_δb) +
                               dot(Jacobi[l, (2lg+2lb+1):(2lg+2lb+ll)], d_pbbf) + 
                               dot(Jacobi[l, (2lg+2lb+ll+1):(2lg+2lb+2ll)], d_pbbt) +
                               dot(Jacobi[l, (2lg+2lb+2ll+1):(2lg+2lb+3ll)], d_qbbf) +
                               dot(Jacobi[l, (2lg+2lb+3ll+1):(2lg+2lb+4ll)], d_qbbt) <= 0)
        
        # real power min limit
        @constraint(m, p_min[g in (2lb+4ll+1+2therm_lim_ctr):(2lb+4ll+2therm_lim_ctr+lg)], gg[g] + dot(Jacobi[g, 1:lg],d_pg) +
                               dot(Jacobi[g, lg+1:2lg],d_qg) +
                               dot(Jacobi[g, 2lg+1:(2lg+lb)], d_vb) +
                               dot(Jacobi[g, (2lg+lb+1):(2lg+2lb)],d_δb) +
                               dot(Jacobi[g, (2lg+2lb+1):(2lg+2lb+ll)], d_pbbf) + 
                               dot(Jacobi[g, (2lg+2lb+ll+1):(2lg+2lb+2ll)], d_pbbt) +
                               dot(Jacobi[g, (2lg+2lb+2ll+1):(2lg+2lb+3ll)], d_qbbf) +
                               dot(Jacobi[g, (2lg+2lb+3ll+1):(2lg+2lb+4ll)], d_qbbt) <= 0) 


        #real power max limit
        @constraint(m, p_max[g in (2lb+4ll+1+2therm_lim_ctr+lg):(2lb+4ll+2therm_lim_ctr+2lg)], gg[g] + dot(Jacobi[g, 1:lg],d_pg) +
                               dot(Jacobi[g, lg+1:2lg],d_qg) +
                               dot(Jacobi[g, 2lg+1:(2lg+lb)], d_vb) +
                               dot(Jacobi[g, (2lg+lb+1):(2lg+2lb)],d_δb) +
                               dot(Jacobi[g, (2lg+2lb+1):(2lg+2lb+ll)], d_pbbf) + 
                               dot(Jacobi[g, (2lg+2lb+ll+1):(2lg+2lb+2ll)], d_pbbt) +
                               dot(Jacobi[g, (2lg+2lb+2ll+1):(2lg+2lb+3ll)], d_qbbf) +
                               dot(Jacobi[g, (2lg+2lb+3ll+1):(2lg+2lb+4ll)], d_qbbt) <= 0)

        #reactive power min limit
        @constraint(m, q_min[g in (2lb+4ll+1+2therm_lim_ctr+2lg):(2lb+4ll+2therm_lim_ctr+3lg)], gg[g] + dot(Jacobi[g, 1:lg],d_pg) +
                               dot(Jacobi[g, lg+1:2lg],d_qg) +
                               dot(Jacobi[g, 2lg+1:(2lg+lb)], d_vb) +
                               dot(Jacobi[g, (2lg+lb+1):(2lg+2lb)],d_δb) +
                               dot(Jacobi[g, (2lg+2lb+1):(2lg+2lb+ll)], d_pbbf) + 
                               dot(Jacobi[g, (2lg+2lb+ll+1):(2lg+2lb+2ll)], d_pbbt) +
                               dot(Jacobi[g, (2lg+2lb+2ll+1):(2lg+2lb+3ll)], d_qbbf) +
                               dot(Jacobi[g, (2lg+2lb+3ll+1):(2lg+2lb+4ll)], d_qbbt) <= 0)
        
        #reactive power max limit
        @constraint(m, q_max[g in (2lb+4ll+1+2therm_lim_ctr+3lg):(2lb+4ll+2therm_lim_ctr+4lg)], gg[g] + dot(Jacobi[g, 1:lg],d_pg) +
                               dot(Jacobi[g, lg+1:2lg],d_qg) +
                               dot(Jacobi[g, 2lg+1:(2lg+lb)], d_vb) +
                               dot(Jacobi[g, (2lg+lb+1):(2lg+2lb)],d_δb) +
                               dot(Jacobi[g, (2lg+2lb+1):(2lg+2lb+ll)], d_pbbf) + 
                               dot(Jacobi[g, (2lg+2lb+ll+1):(2lg+2lb+2ll)], d_pbbt) +
                               dot(Jacobi[g, (2lg+2lb+2ll+1):(2lg+2lb+3ll)], d_qbbf) +
                               dot(Jacobi[g, (2lg+2lb+3ll+1):(2lg+2lb+4ll)], d_qbbt) <= 0)
        
        #voltage level min limit
        @constraint(m, v_min[b in (2lb+4ll+1+2therm_lim_ctr+4lg):(3lb+4ll+2therm_lim_ctr+4lg)], gg[b] + dot(Jacobi[b, 1:lg],d_pg) +
                               dot(Jacobi[b, lg+1:2lg],d_qg) +
                               dot(Jacobi[b, 2lg+1:(2lg+lb)], d_vb) +
                               dot(Jacobi[b, (2lg+lb+1):(2lg+2lb)],d_δb) +
                               dot(Jacobi[b, (2lg+2lb+1):(2lg+2lb+ll)], d_pbbf) + 
                               dot(Jacobi[b, (2lg+2lb+ll+1):(2lg+2lb+2ll)], d_pbbt) +
                               dot(Jacobi[b, (2lg+2lb+2ll+1):(2lg+2lb+3ll)], d_qbbf) +
                               dot(Jacobi[b, (2lg+2lb+3ll+1):(2lg+2lb+4ll)], d_qbbt) <= 0)
        
        #voltage level max limit
        @constraint(m, v_max[b in (3lb+4ll+1+2therm_lim_ctr+4lg):(4lb+4ll+2therm_lim_ctr+4lg)], gg[b] + dot(Jacobi[b, 1:lg],d_pg) +
                               dot(Jacobi[b, lg+1:2lg],d_qg) +
                               dot(Jacobi[b, 2lg+1:(2lg+lb)], d_vb) +
                               dot(Jacobi[b, (2lg+lb+1):(2lg+2lb)],d_δb) +
                               dot(Jacobi[b, (2lg+2lb+1):(2lg+2lb+ll)], d_pbbf) + 
                               dot(Jacobi[b, (2lg+2lb+ll+1):(2lg+2lb+2ll)], d_pbbt) +
                               dot(Jacobi[b, (2lg+2lb+2ll+1):(2lg+2lb+3ll)], d_qbbf) +
                               dot(Jacobi[b, (2lg+2lb+3ll+1):(2lg+2lb+4ll)], d_qbbt) <= 0)

        #slack bus voltage phase 
        @constraint(m, sb[r = (4lb+4ll+1+2therm_lim_ctr+4lg)], gg[r] + dot(Jacobi[r, 1:lg],d_pg) +
                               dot(Jacobi[r, lg+1:2lg],d_qg) +
                               dot(Jacobi[r, 2lg+1:(2lg+lb)], d_vb) +
                               dot(Jacobi[r, (2lg+lb+1):(2lg+2lb)],d_δb) +
                               dot(Jacobi[r, (2lg+2lb+1):(2lg+2lb+ll)], d_pbbf) + 
                               dot(Jacobi[r, (2lg+2lb+ll+1):(2lg+2lb+2ll)], d_pbbt) +
                               dot(Jacobi[r, (2lg+2lb+2ll+1):(2lg+2lb+3ll)], d_qbbf) +
                               dot(Jacobi[r, (2lg+2lb+3ll+1):(2lg+2lb+4ll)], d_qbbt) == 0)
        
        @assert size(Jacobi, 1) == (4lb+4ll+1+2therm_lim_ctr+4lg)
     
        # trust region constraints        
        @constraint(m, d_pg .<= ρ)
        @constraint(m, d_qg .<= ρ)
        @constraint(m, d_vb .<= ρ)
        @constraint(m, d_δb .<= ρ)
        @constraint(m, d_pbbf .<= ρ)
        @constraint(m, d_pbbt .<= ρ)
        @constraint(m, d_qbbf .<= ρ)
        @constraint(m, d_qbbt .<= ρ)
        @constraint(m, d_pg .>= -ρ)
        @constraint(m, d_qg .>= -ρ)
        @constraint(m, d_vb .>= -ρ)
        @constraint(m, d_δb .>= -ρ)
        @constraint(m, d_pbbf .>= -ρ)
        @constraint(m, d_pbbt .>= -ρ)
        @constraint(m, d_qbbf .>= -ρ)
        @constraint(m, d_pbbt .>= -ρ)
  
   
        println("Iteration $i...")
        optimize!(m)
        @show termination_status(m)
        @show objective_value(m)
        
        # update λ with dual values
        λk=[]
        for b in B
            dual = shadow_price(kcl_p[b])
            λk = push!(λk, dual)
        end
        for b in (lb+1):2lb
            dual = shadow_price(kcl_q[b])
            λk = push!(λk, dual)
        end
        for l in (2lb+1):(2lb+ll)
            dual = shadow_price(kvl_pf[l])
            λk = push!(λk, dual)
        end
        for l in (2lb+ll+1):(2lb+2ll)
            dual = shadow_price(kvl_pt[l])
            λk = push!(λk, dual)
        end
        for l in (2lb+2ll+1):(2lb+3ll) 
            dual = shadow_price(kvl_qf[l])
            λk = push!(λk, dual)
        end
        for l in (2lb+3ll+1):(2lb+4ll)
            dual = shadow_price(kvl_qt[l])
            λk = push!(λk, dual)
        end        
        for l in (2lb+4ll+1):(2lb+4ll+therm_lim_ctr)
            dual = shadow_price(tl_f[l])
            λk = push!(λk, dual)
        end
        for l in (2lb+4ll+1+therm_lim_ctr):(2lb+4ll+2therm_lim_ctr)
            dual = shadow_price(tl_t[l])
            λk = push!(λk, dual)
        end
        for g in (2lb+4ll+1+2therm_lim_ctr):(2lb+4ll+2therm_lim_ctr+lg)
            dual = shadow_price(p_min[g])
            λk = push!(λk, dual)
        end
        for g in (2lb+4ll+1+2therm_lim_ctr+lg):(2lb+4ll+2therm_lim_ctr+2lg)
            dual = shadow_price(p_max[g])
            λk = push!(λk, dual)
        end
        for g in (2lb+4ll+1+2therm_lim_ctr+2lg):(2lb+4ll+2therm_lim_ctr+3lg)
            dual = shadow_price(q_min[g])
            λk = push!(λk, dual)
        end
        for g in (2lb+4ll+1+2therm_lim_ctr+3lg):(2lb+4ll+2therm_lim_ctr+4lg)
            dual = shadow_price(q_max[g])
            λk = push!(λk, dual)
        end
        for b in (2lb+4ll+1+2therm_lim_ctr+4lg):(3lb+4ll+2therm_lim_ctr+4lg)
            dual = shadow_price(v_min[b])
            λk = push!(λk, dual)
        end
        for b in (3lb+4ll+1+2therm_lim_ctr+4lg):(4lb+4ll+2therm_lim_ctr+4lg)
            dual = shadow_price(v_max[b])
            λk = push!(λk, dual)
        end
        for i = (4lb+4ll+1+2therm_lim_ctr+4lg)
            dual = shadow_price(sb[i])
            λk = push!(λk, dual)
        end
         λk = convert(Vector{Float64},λk)

        
        # updating difference values
        prop_d_pg = pk .+ value.(d_pg)
        prop_d_qg = qk .+ value.(d_qg)
        prop_d_vb = vk .+ value.(d_vb)
        prop_d_δb = δk .+ value.(d_δb)      
        
        prop_d_pbbf = []
        prop_d_qbbf = []
        prop_d_pbbt = []
        prop_d_qbbt = []
        ctr = 0
        for b1 in B
            for b2 in E_b_f[b1]
                ctr +=1
                prop_d_pbbf = push!(prop_d_pbbf, pbbfk[ctr] + value.(d_pbbf[b1, b2]))
                prop_d_qbbf = push!(prop_d_qbbf, qbbfk[ctr] + value.(d_qbbf[b1, b2]))
                prop_d_pbbt = push!(prop_d_pbbt, pbbtk[ctr] + value.(d_pbbt[b1, b2]))
                prop_d_qbbt = push!(prop_d_qbbt, qbbtk[ctr] + value.(d_qbbt[b1, b2]))
            end
        end
    
        
        # convert all new points into vectors of float64 types
        prop_d_pg = convert(Vector{Float64},prop_d_pg)
        prop_d_qg = convert(Vector{Float64},prop_d_qg)
        prop_d_vb = convert(Vector{Float64},prop_d_vb)
        prop_d_δb = convert(Vector{Float64},prop_d_δb)
        prop_d_pbbf = convert(Vector{Float64},prop_d_pbbf)
        prop_d_pbbt = convert(Vector{Float64},prop_d_pbbt)
        prop_d_qbbf = convert(Vector{Float64},prop_d_qbbf)
        prop_d_qbbt = convert(Vector{Float64},prop_d_qbbt)

        prop_vals = [prop_d_pg;prop_d_qg;prop_d_vb;prop_d_δb;prop_d_pbbf;prop_d_pbbt;prop_d_qbbf;prop_d_qbbt]
 
        
        # merit function
        function mer(x::Vector) 
            newobjev = MOI.eval_objective(d, x)
            ce = zeros(length(all_nonlinear_constraints(nlmod)))
            MOI.eval_constraint(d,ce,x)
            
            eq_rows = ce[1:(2lb+4ll)]
            eq_rows = vcat(eq_rows, ce[end])
            ineq_rows = ce[(2lb+4ll+1):end-1]
            @assert length([eq_rows;ineq_rows])==length(ce)
            
            max_ineq_rows = []
            for i in 1:length(ineq_rows)
                maxval = max(ineq_rows[i], 0)
                max_ineq_rows = push!(max_ineq_rows, maxval)
            end
            
            merit = newobjev + γ * (norm(eq_rows) + norm(max_ineq_rows))
            return merit
        end
        
        vals = Float64.(vals)
        println("m(xk) = $(mer(vals))")
        println("m(prop_xk) = $(mer(prop_vals))")
        
        if termination_status(m) == MathOptInterface.LOCALLY_INFEASIBLE
        elseif mer(prop_vals) < mer(vals)
            println("merit condition: success")
            pk =prop_d_pg
            qk =prop_d_qg
            vk = prop_d_vb
            δk =prop_d_δb
            pbbfk = prop_d_pbbf
            pbbtk = prop_d_pbbt
            qbbfk = prop_d_qbbf
            qbbtk = prop_d_qbbt
            ρ += 1*ρ
        else
            println("merit condition: failed")
            ρ -= 0.5*ρ
        end
        
        #restoration phase:
        if termination_status(m) == MathOptInterface.LOCALLY_INFEASIBLE || termination_status(m) == MathOptInterface.NUMERICAL_ERROR
            pk, qk, vk, δk, pbbfk, pbbtk, qbbfk, qbbtk, ρ = restoration_phase(pk, qk, vk, δk, pbbfk, pbbtk, qbbfk, qbbtk, ρ)
        end
           
        diff_vars = vcat(value.(d_pg), value.(d_qg), value.(d_vb), value.(d_δb), [value.(d_pbbf[b1,b2]) for b1 in B for b2 in E_b_f[b1]], [value.(d_pbbt[b1,b2]) for b1 in B for b2 in E_b_f[b1]], [value.(d_qbbf[b1,b2]) for b1 in B for b2 in E_b_f[b1]], [value.(d_qbbt[b1,b2]) for b1 in B for b2 in E_b_f[b1]])
        
        #convergence test
        tol = 1e-6
        if all(y->(y<=tol), diff_vars)
            @show ρ
            break
        end
    end
end

SQP_MF_OPF (generic function with 1 method)

In [22]:
pk = [1,1,1]
qk=[1,1,1]
vk=[1,1,1,1,1,1,1,1,1]
δk=[0,0,0,0,0,0,0,0,0]
pbbfk=[1,1,1,1,1,1,1,1,1]
pbbtk=[1,1,1,1,1,1,1,1,1]
qbbfk=[1,1,1,1,1,1,1,1,1]
qbbtk=[1,1,1,1,1,1,1,1,1]
λk = zeros(103)
SQP_MF_OPF(pk, qk, vk, δk, pbbfk, pbbtk, qbbfk, qbbtk, λk, 4, 1; maxiter=40)

Iteration 1...
This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:      199
Number of nonzeros in inequality constraint Jacobian.:      186
Number of nonzeros in Lagrangian Hessian.............:        3

Total number of variables............................:       60
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:       55
Total number of inequality constraints...............:      168
        inequality constraints with only lower bounds:       60
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:      108

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  4.9800000e+03 3.25e+00 9.37e-01  -1.0 0.00e+00    -  0.00

┌ Warning: Axis contains one element: 103. If intended, you can safely ignore this warning. To explicitly pass the axis with one element, pass `[103]` instead of `103`.
└ @ JuMP.Containers C:\Users\charl\.julia\packages\JuMP\Y4piv\src\Containers\DenseAxisArray.jl:166



Iteration 2...

┌ Warning: Axis contains one element: 103. If intended, you can safely ignore this warning. To explicitly pass the axis with one element, pass `[103]` instead of `103`.
└ @ JuMP.Containers C:\Users\charl\.julia\packages\JuMP\Y4piv\src\Containers\DenseAxisArray.jl:166



This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:      199
Number of nonzeros in inequality constraint Jacobian.:      186
Number of nonzeros in Lagrangian Hessian.............:       99

Total number of variables............................:       60
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:       55
Total number of inequality constraints...............:      168
        inequality constraints with only lower bounds:       60
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:      108

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  4.9800000e+03 3.25e+00 9.37e-01  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

┌ Warning: Axis contains one element: 103. If intended, you can safely ignore this warning. To explicitly pass the axis with one element, pass `[103]` instead of `103`.
└ @ JuMP.Containers C:\Users\charl\.julia\packages\JuMP\Y4piv\src\Containers\DenseAxisArray.jl:166



This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:      223
Number of nonzeros in inequality constraint Jacobian.:      186
Number of nonzeros in Lagrangian Hessian.............:      102

Total number of variables............................:       60
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:       55
Total number of inequality constraints...............:      168
        inequality constraints with only lower bounds:       60
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:      108

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  4.9800002e+03 2.45e+00 1.09e+00  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

┌ Warning: Axis contains one element: 103. If intended, you can safely ignore this warning. To explicitly pass the axis with one element, pass `[103]` instead of `103`.
└ @ JuMP.Containers C:\Users\charl\.julia\packages\JuMP\Y4piv\src\Containers\DenseAxisArray.jl:166
┌ Warning: Axis contains one element: 103. If intended, you can safely ignore this warning. To explicitly pass the axis with one element, pass `[103]` instead of `103`.
└ @ JuMP.Containers C:\Users\charl\.julia\packages\JuMP\Y4piv\src\Containers\DenseAxisArray.jl:166


This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:      223
Number of nonzeros in inequality constraint Jacobian.:      186
Number of nonzeros in Lagrangian Hessian.............:      102

Total number of variables............................:       60
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:       55
Total number of inequality constraints...............:      168
        inequality constraints with only lower bounds:       60
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:      108

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  4.9800002e+03 2.45e+00 1.09e+00  -1.0 0.00e+00    -  0.00e+00 0.00e+00  

LoadError: InterruptException:

# SQP with a filter function

In [16]:
function SQP_F_OPF(pk, qk, vk, δk, pbbfk, pbbtk, qbbfk, qbbtk, λk, ρ; maxiter=100)

    is_converged = false
    filt = []
    
    # nonlinear model for moi evaluation
    nlmod = Model()
    set_string_names_on_creation(nlmod, false)
    set_silent(nlmod)
    
    @variable(nlmod, pg[g in G]) # real power generation
    @variable(nlmod, qg[g in G]) # reactive power generation
    @variable(nlmod, vb[b in B],(start=1.0)) # bus voltage
    @variable(nlmod, δb[b in B],(start=0.0)) # bus voltage phase
    @variable(nlmod, pbbf[b1 in B, b2 in E_b_f[b1]]) # line real power (from)
    @variable(nlmod, pbbt[b1 in B, b2 in E_b_f[b1]]) # line real power (to)
    @variable(nlmod, qbbf[b1 in B, b2 in E_b_f[b1]]) # line reactive power (from)
    @variable(nlmod, qbbt[b1 in B, b2 in E_b_f[b1]]) # line reactive power (to)

    
    @NLobjective(nlmod, Min, sum(100^2*gen_c[g][1] * pg[g]^2 + 100*gen_c[g][2] * pg[g] + gen_c[g][3] for g in G))
    
    # kcl for real power
    for b in B
        @NLconstraint(nlmod, sum(pg[g] for g in β_g[b]) == demand_real[b] +
                                                           sum(pbbf[b,b_] for b_ in E_b_f[b]) +
                                                           sum(pbbt[b_,b] for b_ in E_b_t[b]) +
                                                           shunt_conductance[b] * (vb[b])^2)
    end
    
    # kcl for reactive power
    for b in B
        @NLconstraint(nlmod, sum(qg[g] for g in β_g[b]) == demand_reactive[b] + sum(qbbf[b,b_] for b_ in E_b_f[b]) +
                                sum(qbbt[b_,b] for b_ in E_b_t[b]) - shunt_susceptance[b] * (vb[b])^2)
    end
    

    # kvl real power from
    for b1 in B
        for b2 in E_b_f[b1]
            @NLconstraint(nlmod, line_con[(b1,b2)]* (vb[b1])^2 - vb[b1]*vb[b2]*
                    (line_con[(b1,b2)] * cos(δb[b1] - δb[b2]) + line_sus[(b1,b2)] * sin(δb[b1] - δb[b2])) == pbbf[b1, b2])
        end
    end
    
    # kvl real power to
    for b1 in B
        for b2 in E_b_f[b1]
            @NLconstraint(nlmod, line_con[(b2,b1)]* (vb[b2])^2 - vb[b2]*vb[b1]*
                    (line_con[(b2,b1)] * cos(δb[b2] - δb[b1]) + line_sus[(b2,b1)] * sin(δb[b2] - δb[b1])) == pbbt[b1, b2])
        end
    end
    
    # kvl reactive power from
    for b1 in B
        for b2 in E_b_f[b1]
            @NLconstraint(nlmod, -line_sus[(b1,b2)]* (vb[b1])^2 - vb[b1]*vb[b2]*
                        (line_con[(b1,b2)] * sin(δb[b1] - δb[b2]) - line_sus[(b1,b2)] * cos(δb[b1] - δb[b2])) == qbbf[b1, b2])
        end
    end
        
    # kvl reactive power to
    for b1 in B
        for b2 in E_b_f[b1]
            @NLconstraint(nlmod, -line_sus[(b2,b1)]* (vb[b2])^2 - vb[b2]*vb[b1]*
                        (line_con[(b2,b1)] * sin(δb[b2] - δb[b1]) - line_sus[(b2,b1)] * cos(δb[b2] - δb[b1])) == qbbt[b1, b2])
        end
    end
    
    # thermal limits from
    therm_lim_ctr = 0
    for b1 in B
        for b2 in E_b_f[b1]
            if line_therm_lim[(b1, b2)] != 0.0
                @NLconstraint(nlmod, (pbbf[b1, b2])^2 + (qbbf[b1, b2])^2 <= (0.01*line_therm_lim[(b1, b2)])^2)
                therm_lim_ctr += 1
            end
        end   
    end
    
    # thermal limits to
    for b1 in B
        for b2 in E_b_f[b1]
            if line_therm_lim[(b1, b2)] != 0.0
                @NLconstraint(nlmod, (pbbt[b1, b2])^2 + (qbbt[b1, b2])^2 <= (0.01*line_therm_lim[(b1, b2)])^2)
            end
        end
    end
    
   # real power min limit
    for g in G
        @NLconstraint(nlmod, pg_minus[g] - pg[g] <= 0) 
    end

    #real power max limit
    for g in G
        @NLconstraint(nlmod, pg[g] <= pg_plus[g])
    end

    #reactive power min limit
    for g in G
        @NLconstraint(nlmod, qg_minus[g] - qg[g] <= 0)
    end

    #reactive power max limit
    for g in G
        @NLconstraint(nlmod, qg[g] <= qg_plus[g])
    end
    
    #voltage level min limit
    for b in B
        @NLconstraint(nlmod, vb_minus[b] - vb[b] <= 0)
    end

    #voltage level max limit
    for b in B
        @NLconstraint(nlmod, vb[b] <= vb_plus[b])
    end

    #slack bus voltage phase 
    for i in 1:length(bus_types)
        if bus_types[i] == 3
            @NLconstraint(nlmod, δb[i] == 0)
        end
    end
    

    d = NLPEvaluator(nlmod)
    MOI.initialize(d, [:Jac,:Grad,:Hess])
    
    for i in 1:maxiter
        if is_converged
            break
        end
        
        d = NLPEvaluator(nlmod)
        MOI.initialize(d, [:Jac,:Grad,:Hess])
        
        vals = [pk; qk; vk; δk; pbbfk; pbbtk; qbbfk; qbbtk]
        
        #Objective eval
        objEv=MOI.eval_objective(d, pk)

        #Objective gradient
        df=zeros(length(pk))
        MOI.eval_objective_gradient(d,df,pk)

        #Constraint eval
        gg = zeros(length(all_nonlinear_constraints(nlmod)))
        MOI.eval_constraint(d,gg,vals)
        
        #Jacobian eval
        JStr=MOI.jacobian_structure(d)
        J=zeros(length(JStr))
        MOI.eval_constraint_jacobian(d,J,vals)

        jr = Vector{Int64}(undef, length(J)); jc = Vector{Int64}(undef, length(J))
        for i in 1:length(JStr)
            jr[i] = JStr[i][1] |> Int
            jc[i] = JStr[i][2] |> Int
        end
        Jacobeval = sparse(jr, jc, J) 
        Jacobi = Matrix(Jacobeval)
        
        #Hessian-of-the-Lagrangian eval
        HStr=MOI.hessian_lagrangian_structure(d)
        H=zeros(length(HStr))
        MOI.eval_hessian_lagrangian(d,H,Float64.(vals),1.0,λk)
 
        
        hr = Vector{Int64}(undef, length(H)); hc = Vector{Int64}(undef, length(H))
        for i in 1:length(HStr)
            hr[i] = HStr[i][1] |> Int
            hc[i] = HStr[i][2] |> Int
        end
        
        HessLag = sparse(hr, hc, H)
        
        
        # linear model    
        m = Model(Ipopt.Optimizer)
#         m = Model(Gurobi.Optimizer)
#         set_silent(m)
        @variable(m, d_pg[g in G])
        @variable(m, d_qg[g in G])
        @variable(m, d_vb[b in B])
        @variable(m, d_δb[b in B])
        @variable(m, d_pbbf[b1 in B, b2 in E_b_f[b1]])
        @variable(m, d_pbbt[b1 in B, b2 in E_b_f[b1]])
        @variable(m, d_qbbf[b1 in B, b2 in E_b_f[b1]])
        @variable(m, d_qbbt[b1 in B, b2 in E_b_f[b1]])
        
        ds = vcat(d_pg, d_qg, d_vb, d_δb, [d_pbbf[b1,b2] for b1 in B for b2 in E_b_f[b1]], [d_pbbt[b1,b2] for b1 in B for b2 in E_b_f[b1]], [d_qbbf[b1,b2] for b1 in B for b2 in E_b_f[b1]], [d_qbbt[b1,b2] for b1 in B for b2 in E_b_f[b1]])
        
        @objective(m, Min, objEv + df'*d_pg + 0.5*ds'*(HessLag*ds)) 
        
        #kcl for real power:
        @constraint(m, kcl_p[b in B], gg[b] + dot(Jacobi[b, 1:lg],d_pg) +
                               dot(Jacobi[b, lg+1:2lg],d_qg) +
                               dot(Jacobi[b, 2lg+1:(2lg+lb)], d_vb) +
                               dot(Jacobi[b, (2lg+lb+1):(2lg+2lb)],d_δb) +
                               dot(Jacobi[b, (2lg+2lb+1):(2lg+2lb+ll)], d_pbbf) + 
                               dot(Jacobi[b, (2lg+2lb+ll+1):(2lg+2lb+2ll)], d_pbbt) +
                               dot(Jacobi[b, (2lg+2lb+2ll+1):(2lg+2lb+3ll)], d_qbbf) +
                               dot(Jacobi[b, (2lg+2lb+3ll+1):(2lg+2lb+4ll)], d_qbbt) == 0)
        

        #kcl for reactive power
        @constraint(m, kcl_q[b in (lb+1):2lb], gg[b] + dot(Jacobi[b, 1:lg],d_pg) +
                               dot(Jacobi[b, lg+1:2lg],d_qg) +
                               dot(Jacobi[b, 2lg+1:(2lg+lb)], d_vb) +
                               dot(Jacobi[b, (2lg+lb+1):(2lg+2lb)],d_δb) +
                               dot(Jacobi[b, (2lg+2lb+1):(2lg+2lb+ll)], d_pbbf) + 
                               dot(Jacobi[b, (2lg+2lb+ll+1):(2lg+2lb+2ll)], d_pbbt) +
                               dot(Jacobi[b, (2lg+2lb+2ll+1):(2lg+2lb+3ll)], d_qbbf) +
                               dot(Jacobi[b, (2lg+2lb+3ll+1):(2lg+2lb+4ll)], d_qbbt) == 0)


        #kvl real power from
        @constraint(m, kvl_pf[l in (2lb+1):(2lb+ll)], gg[l] + dot(Jacobi[l, 1:lg],d_pg) +
                               dot(Jacobi[l, lg+1:2lg],d_qg) +
                               dot(Jacobi[l, 2lg+1:(2lg+lb)], d_vb) +
                               dot(Jacobi[l, (2lg+lb+1):(2lg+2lb)],d_δb) +
                               dot(Jacobi[l, (2lg+2lb+1):(2lg+2lb+ll)], d_pbbf) + 
                               dot(Jacobi[l, (2lg+2lb+ll+1):(2lg+2lb+2ll)], d_pbbt) +
                               dot(Jacobi[l, (2lg+2lb+2ll+1):(2lg+2lb+3ll)], d_qbbf) +
                               dot(Jacobi[l, (2lg+2lb+3ll+1):(2lg+2lb+4ll)], d_qbbt) == 0)
        
                
        #kvl real power to
        @constraint(m, kvl_pt[l in (2lb+ll+1):(2lb+2ll)], gg[l] + dot(Jacobi[l, 1:lg],d_pg) +
                               dot(Jacobi[l, lg+1:2lg],d_qg) +
                               dot(Jacobi[l, 2lg+1:(2lg+lb)], d_vb) +
                               dot(Jacobi[l, (2lg+lb+1):(2lg+2lb)],d_δb) +
                               dot(Jacobi[l, (2lg+2lb+1):(2lg+2lb+ll)], d_pbbf) + 
                               dot(Jacobi[l, (2lg+2lb+ll+1):(2lg+2lb+2ll)], d_pbbt) +
                               dot(Jacobi[l, (2lg+2lb+2ll+1):(2lg+2lb+3ll)], d_qbbf) +
                               dot(Jacobi[l, (2lg+2lb+3ll+1):(2lg+2lb+4ll)], d_qbbt) == 0)
        
        #kvl reactive power from
        @constraint(m, kvl_qf[l in (2lb+2ll+1):(2lb+3ll)], gg[l] + dot(Jacobi[l, 1:lg],d_pg) +
                               dot(Jacobi[l, lg+1:2lg],d_qg) +
                               dot(Jacobi[l, 2lg+1:(2lg+lb)], d_vb) +
                               dot(Jacobi[l, (2lg+lb+1):(2lg+2lb)],d_δb) +
                               dot(Jacobi[l, (2lg+2lb+1):(2lg+2lb+ll)], d_pbbf) + 
                               dot(Jacobi[l, (2lg+2lb+ll+1):(2lg+2lb+2ll)], d_pbbt) +
                               dot(Jacobi[l, (2lg+2lb+2ll+1):(2lg+2lb+3ll)], d_qbbf) +
                               dot(Jacobi[l, (2lg+2lb+3ll+1):(2lg+2lb+4ll)], d_qbbt) == 0)
        
        #kvl reactive power to
        @constraint(m, kvl_qt[l in (2lb+3ll+1):(2lb+4ll)], gg[l] + dot(Jacobi[l, 1:lg],d_pg) +
                               dot(Jacobi[l, lg+1:2lg],d_qg) +
                               dot(Jacobi[l, 2lg+1:(2lg+lb)], d_vb) +
                               dot(Jacobi[l, (2lg+lb+1):(2lg+2lb)],d_δb) +
                               dot(Jacobi[l, (2lg+2lb+1):(2lg+2lb+ll)], d_pbbf) + 
                               dot(Jacobi[l, (2lg+2lb+ll+1):(2lg+2lb+2ll)], d_pbbt) +
                               dot(Jacobi[l, (2lg+2lb+2ll+1):(2lg+2lb+3ll)], d_qbbf) +
                               dot(Jacobi[l, (2lg+2lb+3ll+1):(2lg+2lb+4ll)], d_qbbt) == 0)     
       
        #thermal limits from
        @constraint(m, tl_f[l in (2lb+4ll+1):(2lb+4ll+therm_lim_ctr)], gg[l] + dot(Jacobi[l, 1:lg],d_pg) +
                               dot(Jacobi[l, lg+1:2lg],d_qg) +
                               dot(Jacobi[l, 2lg+1:(2lg+lb)], d_vb) +
                               dot(Jacobi[l, (2lg+lb+1):(2lg+2lb)],d_δb) +
                               dot(Jacobi[l, (2lg+2lb+1):(2lg+2lb+ll)], d_pbbf) + 
                               dot(Jacobi[l, (2lg+2lb+ll+1):(2lg+2lb+2ll)], d_pbbt) +
                               dot(Jacobi[l, (2lg+2lb+2ll+1):(2lg+2lb+3ll)], d_qbbf) +
                               dot(Jacobi[l, (2lg+2lb+3ll+1):(2lg+2lb+4ll)], d_qbbt) <= 0)
        
        #thermal limits to
        @constraint(m, tl_t[l in (2lb+4ll+1+therm_lim_ctr):(2lb+4ll+2therm_lim_ctr)], gg[l] + dot(Jacobi[l, 1:lg],d_pg) +
                               dot(Jacobi[l, lg+1:2lg],d_qg) +
                               dot(Jacobi[l, 2lg+1:(2lg+lb)], d_vb) +
                               dot(Jacobi[l, (2lg+lb+1):(2lg+2lb)],d_δb) +
                               dot(Jacobi[l, (2lg+2lb+1):(2lg+2lb+ll)], d_pbbf) + 
                               dot(Jacobi[l, (2lg+2lb+ll+1):(2lg+2lb+2ll)], d_pbbt) +
                               dot(Jacobi[l, (2lg+2lb+2ll+1):(2lg+2lb+3ll)], d_qbbf) +
                               dot(Jacobi[l, (2lg+2lb+3ll+1):(2lg+2lb+4ll)], d_qbbt) <= 0)
        
        # real power min limit
        @constraint(m, p_min[g in (2lb+4ll+1+2therm_lim_ctr):(2lb+4ll+2therm_lim_ctr+lg)], gg[g] + dot(Jacobi[g, 1:lg],d_pg) +
                               dot(Jacobi[g, lg+1:2lg],d_qg) +
                               dot(Jacobi[g, 2lg+1:(2lg+lb)], d_vb) +
                               dot(Jacobi[g, (2lg+lb+1):(2lg+2lb)],d_δb) +
                               dot(Jacobi[g, (2lg+2lb+1):(2lg+2lb+ll)], d_pbbf) + 
                               dot(Jacobi[g, (2lg+2lb+ll+1):(2lg+2lb+2ll)], d_pbbt) +
                               dot(Jacobi[g, (2lg+2lb+2ll+1):(2lg+2lb+3ll)], d_qbbf) +
                               dot(Jacobi[g, (2lg+2lb+3ll+1):(2lg+2lb+4ll)], d_qbbt) <= 0) 


        #real power max limit
        @constraint(m, p_max[g in (2lb+4ll+1+2therm_lim_ctr+lg):(2lb+4ll+2therm_lim_ctr+2lg)], gg[g] + dot(Jacobi[g, 1:lg],d_pg) +
                               dot(Jacobi[g, lg+1:2lg],d_qg) +
                               dot(Jacobi[g, 2lg+1:(2lg+lb)], d_vb) +
                               dot(Jacobi[g, (2lg+lb+1):(2lg+2lb)],d_δb) +
                               dot(Jacobi[g, (2lg+2lb+1):(2lg+2lb+ll)], d_pbbf) + 
                               dot(Jacobi[g, (2lg+2lb+ll+1):(2lg+2lb+2ll)], d_pbbt) +
                               dot(Jacobi[g, (2lg+2lb+2ll+1):(2lg+2lb+3ll)], d_qbbf) +
                               dot(Jacobi[g, (2lg+2lb+3ll+1):(2lg+2lb+4ll)], d_qbbt) <= 0)

        #reactive power min limit
        @constraint(m, q_min[g in (2lb+4ll+1+2therm_lim_ctr+2lg):(2lb+4ll+2therm_lim_ctr+3lg)], gg[g] + dot(Jacobi[g, 1:lg],d_pg) +
                               dot(Jacobi[g, lg+1:2lg],d_qg) +
                               dot(Jacobi[g, 2lg+1:(2lg+lb)], d_vb) +
                               dot(Jacobi[g, (2lg+lb+1):(2lg+2lb)],d_δb) +
                               dot(Jacobi[g, (2lg+2lb+1):(2lg+2lb+ll)], d_pbbf) + 
                               dot(Jacobi[g, (2lg+2lb+ll+1):(2lg+2lb+2ll)], d_pbbt) +
                               dot(Jacobi[g, (2lg+2lb+2ll+1):(2lg+2lb+3ll)], d_qbbf) +
                               dot(Jacobi[g, (2lg+2lb+3ll+1):(2lg+2lb+4ll)], d_qbbt) <= 0)
        
        #reactive power max limit
        @constraint(m, q_max[g in (2lb+4ll+1+2therm_lim_ctr+3lg):(2lb+4ll+2therm_lim_ctr+4lg)], gg[g] + dot(Jacobi[g, 1:lg],d_pg) +
                               dot(Jacobi[g, lg+1:2lg],d_qg) +
                               dot(Jacobi[g, 2lg+1:(2lg+lb)], d_vb) +
                               dot(Jacobi[g, (2lg+lb+1):(2lg+2lb)],d_δb) +
                               dot(Jacobi[g, (2lg+2lb+1):(2lg+2lb+ll)], d_pbbf) + 
                               dot(Jacobi[g, (2lg+2lb+ll+1):(2lg+2lb+2ll)], d_pbbt) +
                               dot(Jacobi[g, (2lg+2lb+2ll+1):(2lg+2lb+3ll)], d_qbbf) +
                               dot(Jacobi[g, (2lg+2lb+3ll+1):(2lg+2lb+4ll)], d_qbbt) <= 0)
        
        #voltage level min limit
        @constraint(m, v_min[b in (2lb+4ll+1+2therm_lim_ctr+4lg):(3lb+4ll+2therm_lim_ctr+4lg)], gg[b] + dot(Jacobi[b, 1:lg],d_pg) +
                               dot(Jacobi[b, lg+1:2lg],d_qg) +
                               dot(Jacobi[b, 2lg+1:(2lg+lb)], d_vb) +
                               dot(Jacobi[b, (2lg+lb+1):(2lg+2lb)],d_δb) +
                               dot(Jacobi[b, (2lg+2lb+1):(2lg+2lb+ll)], d_pbbf) + 
                               dot(Jacobi[b, (2lg+2lb+ll+1):(2lg+2lb+2ll)], d_pbbt) +
                               dot(Jacobi[b, (2lg+2lb+2ll+1):(2lg+2lb+3ll)], d_qbbf) +
                               dot(Jacobi[b, (2lg+2lb+3ll+1):(2lg+2lb+4ll)], d_qbbt) <= 0)
        
        #voltage level max limit
        @constraint(m, v_max[b in (3lb+4ll+1+2therm_lim_ctr+4lg):(4lb+4ll+2therm_lim_ctr+4lg)], gg[b] + dot(Jacobi[b, 1:lg],d_pg) +
                               dot(Jacobi[b, lg+1:2lg],d_qg) +
                               dot(Jacobi[b, 2lg+1:(2lg+lb)], d_vb) +
                               dot(Jacobi[b, (2lg+lb+1):(2lg+2lb)],d_δb) +
                               dot(Jacobi[b, (2lg+2lb+1):(2lg+2lb+ll)], d_pbbf) + 
                               dot(Jacobi[b, (2lg+2lb+ll+1):(2lg+2lb+2ll)], d_pbbt) +
                               dot(Jacobi[b, (2lg+2lb+2ll+1):(2lg+2lb+3ll)], d_qbbf) +
                               dot(Jacobi[b, (2lg+2lb+3ll+1):(2lg+2lb+4ll)], d_qbbt) <= 0)

        #slack bus voltage phase 
        @constraint(m, sb[r = (4lb+4ll+1+2therm_lim_ctr+4lg)], gg[r] + dot(Jacobi[r, 1:lg],d_pg) +
                               dot(Jacobi[r, lg+1:2lg],d_qg) +
                               dot(Jacobi[r, 2lg+1:(2lg+lb)], d_vb) +
                               dot(Jacobi[r, (2lg+lb+1):(2lg+2lb)],d_δb) +
                               dot(Jacobi[r, (2lg+2lb+1):(2lg+2lb+ll)], d_pbbf) + 
                               dot(Jacobi[r, (2lg+2lb+ll+1):(2lg+2lb+2ll)], d_pbbt) +
                               dot(Jacobi[r, (2lg+2lb+2ll+1):(2lg+2lb+3ll)], d_qbbf) +
                               dot(Jacobi[r, (2lg+2lb+3ll+1):(2lg+2lb+4ll)], d_qbbt) == 0)
        
        @assert size(Jacobi, 1) == (4lb+4ll+1+2therm_lim_ctr+4lg)
     
        # trust region constraints        
        @constraint(m, d_pg .<= ρ)
        @constraint(m, d_qg .<= ρ)
        @constraint(m, d_vb .<= ρ)
        @constraint(m, d_δb .<= ρ)
        @constraint(m, d_pbbf .<= ρ)
        @constraint(m, d_pbbt .<= ρ)
        @constraint(m, d_qbbf .<= ρ)
        @constraint(m, d_qbbt .<= ρ)
        @constraint(m, d_pg .>= -ρ)
        @constraint(m, d_qg .>= -ρ)
        @constraint(m, d_vb .>= -ρ)
        @constraint(m, d_δb .>= -ρ)
        @constraint(m, d_pbbf .>= -ρ)
        @constraint(m, d_pbbt .>= -ρ)
        @constraint(m, d_qbbf .>= -ρ)
        @constraint(m, d_pbbt .>= -ρ)
  
   
        println("Iteration $i...")
        optimize!(m)
        @show termination_status(m)
        @show objective_value(m)
        
        # update λ with dual values
        λk=[]
        for b in B
            dual = shadow_price(kcl_p[b])
            λk = push!(λk, dual)
        end
        for b in (lb+1):2lb
            dual = shadow_price(kcl_q[b])
            λk = push!(λk, dual)
        end
        for l in (2lb+1):(2lb+ll)
            dual = shadow_price(kvl_pf[l])
            λk = push!(λk, dual)
        end
        for l in (2lb+ll+1):(2lb+2ll)
            dual = shadow_price(kvl_pt[l])
            λk = push!(λk, dual)
        end
        for l in (2lb+2ll+1):(2lb+3ll) 
            dual = shadow_price(kvl_qf[l])
            λk = push!(λk, dual)
        end
        for l in (2lb+3ll+1):(2lb+4ll)
            dual = shadow_price(kvl_qt[l])
            λk = push!(λk, dual)
        end        
        for l in (2lb+4ll+1):(2lb+4ll+therm_lim_ctr)
            dual = shadow_price(tl_f[l])
            λk = push!(λk, dual)
        end
        for l in (2lb+4ll+1+therm_lim_ctr):(2lb+4ll+2therm_lim_ctr)
            dual = shadow_price(tl_t[l])
            λk = push!(λk, dual)
        end
        for g in (2lb+4ll+1+2therm_lim_ctr):(2lb+4ll+2therm_lim_ctr+lg)
            dual = shadow_price(p_min[g])
            λk = push!(λk, dual)
        end
        for g in (2lb+4ll+1+2therm_lim_ctr+lg):(2lb+4ll+2therm_lim_ctr+2lg)
            dual = shadow_price(p_max[g])
            λk = push!(λk, dual)
        end
        for g in (2lb+4ll+1+2therm_lim_ctr+2lg):(2lb+4ll+2therm_lim_ctr+3lg)
            dual = shadow_price(q_min[g])
            λk = push!(λk, dual)
        end
        for g in (2lb+4ll+1+2therm_lim_ctr+3lg):(2lb+4ll+2therm_lim_ctr+4lg)
            dual = shadow_price(q_max[g])
            λk = push!(λk, dual)
        end
        for b in (2lb+4ll+1+2therm_lim_ctr+4lg):(3lb+4ll+2therm_lim_ctr+4lg)
            dual = shadow_price(v_min[b])
            λk = push!(λk, dual)
        end
        for b in (3lb+4ll+1+2therm_lim_ctr+4lg):(4lb+4ll+2therm_lim_ctr+4lg)
            dual = shadow_price(v_max[b])
            λk = push!(λk, dual)
        end
        for i = (4lb+4ll+1+2therm_lim_ctr+4lg)
            dual = shadow_price(sb[i])
            λk = push!(λk, dual)
        end
         λk = convert(Vector{Float64},λk)

        
        # updating difference values
        prop_d_pg = pk .+ value.(d_pg)
        prop_d_qg = qk .+ value.(d_qg)
        prop_d_vb = vk .+ value.(d_vb)
        prop_d_δb = δk .+ value.(d_δb)      
        
        prop_d_pbbf = []
        prop_d_qbbf = []
        prop_d_pbbt = []
        prop_d_qbbt = []
        ctr = 0
        for b1 in B
            for b2 in E_b_f[b1]
                ctr +=1
                prop_d_pbbf = push!(prop_d_pbbf, pbbfk[ctr] + value.(d_pbbf[b1, b2]))
                prop_d_qbbf = push!(prop_d_qbbf, qbbfk[ctr] + value.(d_qbbf[b1, b2]))
                prop_d_pbbt = push!(prop_d_pbbt, pbbtk[ctr] + value.(d_pbbt[b1, b2]))
                prop_d_qbbt = push!(prop_d_qbbt, qbbtk[ctr] + value.(d_qbbt[b1, b2]))
            end
        end
    
        
        # convert all new points into vectors of float64 types
        prop_d_pg = convert(Vector{Float64},prop_d_pg)
        prop_d_qg = convert(Vector{Float64},prop_d_qg)
        prop_d_vb = convert(Vector{Float64},prop_d_vb)
        prop_d_δb = convert(Vector{Float64},prop_d_δb)
        prop_d_pbbf = convert(Vector{Float64},prop_d_pbbf)
        prop_d_pbbt = convert(Vector{Float64},prop_d_pbbt)
        prop_d_qbbf = convert(Vector{Float64},prop_d_qbbf)
        prop_d_qbbt = convert(Vector{Float64},prop_d_qbbt)

        prop_vals = [prop_d_pg;prop_d_qg;prop_d_vb;prop_d_δb;prop_d_pbbf;prop_d_pbbt;prop_d_qbbf;prop_d_qbbt]
 
        #functions for making the filter
        function fk(x::Vector{<:Real}) 
            newobjev = MOI.eval_objective(d, x)
            return newobjev
        end
        
        function hk(x::Vector{<:Real}) 
            ce = zeros(length(all_nonlinear_constraints(nlmod)))
            MOI.eval_constraint(d,ce,x)
            
            eq_rows = ce[1:(2lb+4ll)]
            eq_rows = vcat(eq_rows, ce[end])
            ineq_rows = ce[(2lb+4ll+1):end-1]
            @assert length([eq_rows;ineq_rows])==length(ce)
            
            max_ineq_rows = []
            for i in 1:length(ineq_rows)
                maxval = max(ineq_rows[i], 0)
                max_ineq_rows = push!(max_ineq_rows, maxval)
            end
            
            h = norm(eq_rows) + norm(max_ineq_rows)
            return h
        end
        
        vals = Float64.(vals)
        println("f(propxk) = $(fk(prop_vals))")
        println("f(xk) = $(fk(vals))")
        
        better_obj = all(v -> fk(prop_vals) < v[1], filt)
        better_constr = all(v -> hk(prop_vals) < v[2], filt)
        
        diff_vars = vcat(value.(d_pg), value.(d_qg), value.(d_vb), value.(d_δb), [value.(d_pbbf[b1,b2]) for b1 in B for b2 in E_b_f[b1]], [value.(d_pbbt[b1,b2]) for b1 in B for b2 in E_b_f[b1]], [value.(d_qbbf[b1,b2]) for b1 in B for b2 in E_b_f[b1]], [value.(d_qbbt[b1,b2]) for b1 in B for b2 in E_b_f[b1]])
        
        if termination_status(m) == MathOptInterface.LOCALLY_INFEASIBLE
        elseif better_obj || better_constr
            println("filter: accept")
            pk =prop_d_pg
            qk =prop_d_qg
            vk = prop_d_vb
            δk =prop_d_δb
            pbbfk = prop_d_pbbf
            pbbtk = prop_d_pbbt
            qbbfk = prop_d_qbbf
            qbbtk = prop_d_qbbt
            ρ += 1*ρ
            
            vals = [pk; qk; vk; δk; pbbfk; pbbtk; qbbfk; qbbtk]
            vals = Float64.(vals)
            filt = push!(filt, (fk(vals), hk(vals)))
        else
            println("filter: reject")
#             ρ -= 0.5*ρ
            max_step = norm(diff_vars, Inf) # more aggressive trust region
            ρ = max_step/2
            filt = push!(filt, (fk(prop_vals), hk(prop_vals)))
        end
        
        #restoration phase:
        if termination_status(m) == MathOptInterface.LOCALLY_INFEASIBLE || termination_status(m) == MathOptInterface.NUMERICAL_ERROR
            pk, qk, vk, δk, pbbfk, pbbtk, qbbfk, qbbtk, ρ = restoration_phase(pk, qk, vk, δk, pbbfk, pbbtk, qbbfk, qbbtk, ρ)
        end
        
        #convergence test
        tol = 1e-6
        if all(y->(y<=tol), diff_vars)
            @show ρ
            break
        end
                
    end
end

SQP_F_OPF (generic function with 1 method)

In [19]:
pk = [1,1,1]
qk=[1,1,1]
vk=[1,1,1,1,1,1,1,1,1]
δk=[0,0,0,0,0,0,0,0,0]
pbbfk=[1,1,1,1,1,1,1,1,1]
pbbtk=[1,1,1,1,1,1,1,1,1]
qbbfk=[1,1,1,1,1,1,1,1,1]
qbbtk=[1,1,1,1,1,1,1,1,1]
λk = zeros(103)
SQP_F_OPF(pk, qk, vk, δk, pbbfk, pbbtk, qbbfk, qbbtk, λk, 4; maxiter=40)

Iteration 1...

┌ Warning: Axis contains one element: 103. If intended, you can safely ignore this warning. To explicitly pass the axis with one element, pass `[103]` instead of `103`.
└ @ JuMP.Containers C:\Users\charl\.julia\packages\JuMP\Y4piv\src\Containers\DenseAxisArray.jl:166



This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:      199
Number of nonzeros in inequality constraint Jacobian.:      186
Number of nonzeros in Lagrangian Hessian.............:        3

Total number of variables............................:       60
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:       55
Total number of inequality constraints...............:      168
        inequality constraints with only lower bounds:       60
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:      108

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  4.9800000e+03 3.25e+00 9.37e-01  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

┌ Warning: Axis contains one element: 103. If intended, you can safely ignore this warning. To explicitly pass the axis with one element, pass `[103]` instead of `103`.
└ @ JuMP.Containers C:\Users\charl\.julia\packages\JuMP\Y4piv\src\Containers\DenseAxisArray.jl:166



This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:      223
Number of nonzeros in inequality constraint Jacobian.:      186
Number of nonzeros in Lagrangian Hessian.............:      102

Total number of variables............................:       60
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:       55
Total number of inequality constraints...............:      168
        inequality constraints with only lower bounds:       60
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:      108

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  5.2160266e+03 2.56e+00 3.41e-01  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

┌ Warning: Axis contains one element: 103. If intended, you can safely ignore this warning. To explicitly pass the axis with one element, pass `[103]` instead of `103`.
└ @ JuMP.Containers C:\Users\charl\.julia\packages\JuMP\Y4piv\src\Containers\DenseAxisArray.jl:166


restoration merit condition: failed
restoration merit condition: failed
restoration merit condition: success
restoration merit condition: failed
restoration merit condition: failed
restoration merit condition: failed
restoration merit condition: failed
restoration merit condition: failed
restoration merit condition: failed
restoration merit condition: failed
restoration merit condition: failed
restoration merit condition: failed
restoration merit condition: failed
restoration merit condition: failed
restoration merit condition: failed
restoration merit condition: failed
restoration merit condition: failed
restoration merit condition: failed
restoration merit condition: failed
restoration merit condition: failed
restoration merit condition: failed
restoration merit condition: failed
restoration merit condition: failed
restoration merit condition: failed
restoration merit condition: failed
restoration merit condition: failed
restoration merit condition: failed
restoration merit condition

LoadError: InterruptException: